<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/Lectute-2/Lectute_2_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#2. Методы векторизации текстовых данных

## 2.1. One-Hot Encoding

### 2.1.1. Введение

Одной из ключевых задач в обработке естественного языка (Natural Language Processing, NLP) является **представление текста в числовом виде**, пригодном для обработки алгоритмами машинного обучения. Поскольку компьютеры не могут напрямую работать с текстовыми символами, необходимо преобразовать слова, предложения или документы в **векторные представления**.

Наиболее простым и интуитивно понятным способом векторизации категориальных данных, включая слова, является метод **One-Hot Encoding** (кодирование с одним активным состоянием). Данный метод широко используется на начальных этапах изучения NLP как базовая модель преобразования слов в векторы.

В этом разделе мы подробно рассмотрим принцип работы One-Hot Encoding, его математическую основу, практическую реализацию и **критические ограничения**, особенно с точки зрения **объёма занимаемой памяти**. В завершение будет приведён **реалистичный пример масштабного текстового корпуса**, демонстрирующий, почему One-Hot Encoding оказывается **неприемлемым для крупных задач**.



### 2.1.2. Определение и принцип работы

Пусть задан текстовый корпус — совокупность документов (предложений, абзацев, книг), из которого извлекаются все уникальные слова. Совокупность этих слов образует **словарь (vocabulary)**. Обозначим размер словаря как:

$$V = |\text{vocabulary}|$$

Каждому слову $w_i$ в словаре присваивается уникальный индекс $i \in \{0, 1, 2, \dots, V-1\}$.

**One-Hot Encoding** слова $w_i$ — это бинарный вектор $\mathbf{v}_i \in \mathbb{R}^V$, определяемый следующим образом:

$$
v_{i,j} =
\begin{cases}
1, & \text{если } j = i, \\
0, & \text{если } j \ne i.
\end{cases}
$$

Таким образом, вектор имеет длину $V$, и только одна компонента (на позиции $i$) равна 1, а все остальные — 0.



### 2.1.3. Пример построения One-Hot векторов

Рассмотрим небольшой словарь из четырёх слов:

| Слово    | Индекс |
|---------|--------|
| кошка   | 0      |
| собака  | 1      |
| бежит   | 2      |
| спит    | 3      |

Тогда One-Hot векторы будут:

- "кошка" → $[1, 0, 0, 0]$
- "собака" → $[0, 1, 0, 0]$
- "бежит" → $[0, 0, 1, 0]$
- "спит" → $[0, 0, 0, 1]$

Как видно, каждый вектор однозначно идентифицирует слово, но не содержит никакой информации о его значении, контексте или связи с другими словами.



### 2.1.4. Свойства One-Hot представления

| Свойство | Характеристика |
|--------|----------------|
| **Размерность** | Равна размеру словаря $V$ |
| **Разреженность** | Вектор содержит $V-1$ нулей и один 1 → очень высокая разрежённость |
| **Ортогональность** | Все векторы попарно ортогональны: $\mathbf{v}_i \cdot \mathbf{v}_j = 0$ при $i \ne j$ |
| **Семантика** | Не учитывается. Все слова равноудалены в векторном пространстве |
| **OOV-слова** | Слова, не вошедшие в словарь, не могут быть закодированы (исключение — специальные стратегии, например, нулевой вектор) |


### 2.1.5. Практическая реализация

Ниже приведён пример реализации One-Hot Encoding на языке Python с использованием библиотек `scikit-learn` и `numpy`.


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Пример текстов
sentences = [
    "Я люблю кошек",
    "Собаки тоже хорошие"
]

# Токенизация и создание словаря
words = []
for sentence in sentences:
    cleaned = sentence.lower().replace('.', '').replace(',', '').split()
    words.extend(cleaned)

unique_words = sorted(set(words))
print(f"Словарь: {unique_words}")

# Обучение One-Hot Encoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(np.array(unique_words).reshape(-1, 1))

# Функция кодирования слова
def encode_word(word):
    return encoder.transform([[word.lower()]])

# Примеры
print("Вектор 'кошек':", encode_word("кошек"))
print("Вектор 'птицы' (OOV):", encode_word("птицы"))



> **Пояснение параметров**:
> - `handle_unknown='ignore'` — позволяет обрабатывать слова вне словаря, возвращая вектор из нулей.
> - `sparse_output=False` — возвращает плотный массив (удобно для демонстрации).
> - При работе с большими словарями рекомендуется использовать `sparse=True` для экономии памяти.



### 2.1.6. Проблема масштабируемости: анализ объёма памяти

Одним из главных недостатков One-Hot Encoding является **экстремально высокое потребление памяти** при увеличении размера словаря. Рассмотрим это на примере крупного текстового корпуса.

#### 📘 Пример: корпус из 1000 книг

Представим, что у нас есть коллекция из **1000 книг**, каждая объёмом:

- **250 страниц**
- **35 строк на странице**
- **~10 слов в строке**

##### Шаг 1: Общее количество слов

$$1000 \times 250 \times 35 \times 10 = 87\,500\,000$$

Итого: **87.5 миллионов слов**.

##### Шаг 2: Оценка размера словаря

Пусть средний словарь одной книги — около 20 000 уникальных слов. Учитывая пересечение лексики между книгами, общее количество уникальных слов во всём корпусе оценим как:

$$V = 100\,000$$

> Это реалистичная оценка для разнообразного корпуса (художественная и научная литература).

##### Шаг 3: Память на один One-Hot вектор

Каждый вектор имеет длину $V = 100\,000$. Если использовать тип `float32` (4 байта на элемент), то объём на один вектор:

$$100\,000 \times 4 = 400\,000\ \text{байт} = 400\ \text{КБ}$$

##### Шаг 4: Общий объём памяти для всего корпуса

Для кодирования **87.5 млн слов** потребуется:

$$87\,500\,000 \times 400\,000 = 35\,000\,000\,000\,000\ \text{байт} = 35\ \text{ТБ}$$

> 💥 **Итого: 35 терабайт оперативной памяти или дискового пространства.**



### 2.1.7. Анализ результатов

| Параметр | Значение |
|--------|--------|
| Количество слов | 87.5 млн |
| Размер словаря $V$ | 100 000 |
| Память на одно слово | 400 КБ |
| **Общий объём** | **35 ТБ** |

#### Выводы:
- **Один вектор** занимает **400 КБ**, хотя содержит только **одну единицу**.
- **99.999% данных** — это нули, что делает представление крайне **неэффективным**.
- Хранение полных векторов в плотном формате **непрактично даже для средних корпусов**.
- Даже при использовании **разреженных матриц**, где хранятся только индексы единиц, объём можно сократить до:
  $$87\,500\,000 \times 4\ \text{байта (на индекс)} = 350\ \text{МБ}$$
  — но это уже **не векторы отдельных слов**, а сжатое представление.



### 2.1.8. Ограничения метода

На основе проведённого анализа можно выделить следующие **фундаментальные недостатки** One-Hot Encoding:

1. **Неэффективность по памяти**  
   Объём памяти растёт пропорционально $V \times N$, где $N$ — количество слов. Уже при $V > 10^4$ метод становится неприменимым.

2. **Отсутствие семантической информации**  
   Все слова находятся на одинаковом "расстоянии". Например, косинусное расстояние между любыми двумя разными векторами:
   $$\cos(\mathbf{v}_i, \mathbf{v}_j) = 0 \quad \text{при} \quad i \ne j$$
   Это означает, что модель не может отличить близкие по смыслу слова (например, "кошка" и "собака") от совершенно разных ("кошка" и "бежит").

3. **Невозможность обобщения**  
   Метод не учитывает морфологию, синонимы или контекст.

4. **Проблема OOV (Out-of-Vocabulary)**  
   Любое новое слово, не вошедшее в обучающий словарь, не может быть корректно закодировано.



### 2.1.9. Когда можно использовать One-Hot?

Несмотря на ограничения, One-Hot Encoding имеет право на существование в следующих случаях:

- **Малые словари**: например, в задачах классификации с ограниченным числом категорий (например, 10–100 классов).
- **Обучающие цели**: как вводный метод для понимания векторизации.
- **Входной слой нейросетей**: в современных архитектурах One-Hot векторы часто используются неявно — через **embedding-слои**, которые сразу преобразуют индекс слова в плотный вектор.



### 2.1.10. Заключение

One-Hot Encoding — это **простой, но крайне неэффективный** способ векторизации слов. Он служит важным концептуальным шагом в изучении NLP, демонстрируя, как текст можно преобразовать в числовой формат. Однако его применение в реальных задачах с большими объёмами текста **практически исключено** из-за огромных требований к памяти и отсутствия семантической структуры.

Приведённый пример с 1000 книгами показывает, что даже при скромных предположениях объём данных может достигать **десятков терабайт**, что делает метод **непригодным для масштабных приложений**.

В следующих главах мы рассмотрим более эффективные методы векторизации, такие как **Word2Vec**, **GloVe** и **контекстные эмбеддинги**, которые решают эти проблемы за счёт компактности и способности улавливать смысл слов.




# 2.2. Bag-of-Words (мешок слов)

## 2.2.1. Определение и принцип работы

**Bag-of-Words (BoW)** — это классическая модель представления текстовых данных, при которой документ описывается как **неупорядоченный набор (мультимножество) слов**, с учётом их частоты, но **без учёта грамматики и порядка**.

Каждый документ преобразуется в числовой вектор, размерность которого равна размеру словаря $V$, где $V$ — общее количество уникальных слов во всём корпусе. Элемент вектора на позиции $j$ содержит количество вхождений слова $w_j$ в данный документ.

Формально, пусть:
- $\mathcal{D} = \{d_1, d_2, \dots, d_N\}$ — коллекция из $N$ документов,
- $\mathcal{V} = \{w_1, w_2, \dots, w_V\}$ — словарь корпуса,
- $f_{ij}$ — частота слова $w_j$ в документе $d_i$.

Тогда векторное представление документа $d_i$ в модели BoW имеет вид:

$$
\mathbf{v}_i = [f_{i1}, f_{i2}, \dots, f_{iV}]
$$

Такой подход позволяет преобразовать текст в структурированный числовой формат, пригодный для использования в алгоритмах машинного обучения.



## 2.2.2. Пример построения BoW-векторов

Рассмотрим следующие документы:

- Документ 1: *"Кошка бежит"*
- Документ 2: *"Собака спит"*

Построим словарь (лексикографически):

$$
\text{vocabulary} = \{\text{кошка}: 0, \text{собака}: 1, \text{бежит}: 2, \text{спит}: 3\}
$$

Тогда BoW-векторы будут:

- Вектор для Документа 1: $[1, 0, 1, 0]$
- Вектор для Документа 2: $[0, 1, 0, 1]$

> Обратите внимание: в отличие от One-Hot Encoding, где каждый вектор содержит только одну единицу, BoW позволяет **множественные ненулевые значения**, отражающие **реальную частоту слов**.



## 2.2.3. Алгоритм построения BoW

1. **Токенизация**  
   Каждый документ разбивается на отдельные слова (токены), обычно с приведением к нижнему регистру и удалением знаков препинания.

2. **Построение словаря**  
   Из всех токенов корпуса формируется упорядоченный список уникальных слов. Каждому слову присваивается фиксированный индекс.

3. **Векторизация**  
   Для каждого документа строится вектор длины $V$, в котором на позиции $j$ записывается количество вхождений слова $w_j$ в документ.



## 2.2.4. Практическая реализация на Python



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Пример документов
documents = [
    "Кот спит на диване",
    "Собака бежит по улице",
    "Кот и собака играют"
]

# Создаём векторизатор BoW
vectorizer = CountVectorizer()

# Обучаем и преобразуем документы
X_bow = vectorizer.fit_transform(documents)

# Получаем словарь
feature_names = vectorizer.get_feature_names_out()
print(f"Словарь (feature names): {feature_names}")

# Выводим BoW-векторы
print("\nBoW векторы для документов:")
print(X_bow.toarray())

# Пример для нового документа
new_doc = ["Собака спит на коврике"]
new_doc_bow = vectorizer.transform(new_doc)
print(f"\nBoW вектор для нового документа '{new_doc[0]}':")
print(new_doc_bow.toarray())


> **Пояснение**:
> - `CountVectorizer` автоматически обрабатывает текст: токенизирует, приводит к нижнему регистру, строит словарь.
> - Результат `X_bow` — разреженная матрица, что позволяет эффективно хранить большие корпусы.
> - Новые документы векторизуются с использованием **того же словаря**, что и при обучении.



## 2.2.5. Преимущества BoW перед One-Hot Encoding

Несмотря на схожую структуру (оба метода используют векторы длины $V$), **Bag-of-Words значительно превосходит One-Hot Encoding** по нескольким ключевым параметрам.

### 1. **Учёт частоты слов**

- **One-Hot**: каждое слово представлено вектором с одной единицей, независимо от того, сколько раз оно встречается.
- **BoW**: если слово встречается дважды, его счётчик будет равен 2 — это **дополнительная семантическая информация**.

> Например, в документе *"кошка кошка кошка"* слово "кошка" явно играет важную роль. One-Hot не различит его от документа с одним вхождением, а BoW — да.



### 2. **Эффективное представление целых документов**

- **One-Hot**: кодирует **одно слово**.
- **BoW**: кодирует **целый документ** как сумму вкладов всех слов.

> Это делает BoW **готовым к использованию в задачах классификации, кластеризации и поиска**, в то время как One-Hot требует дополнительных шагов (например, суммирования векторов слов).



### 3. **Более информативные векторы**

Рассмотрим два документа:
- $d_1$: *"кошка бежит"*
- $d_2$: *"кошка бежит бежит"*

| Метод | Вектор $d_1$ | Вектор $d_2$ |
|------|--------------|--------------|
| One-Hot (на слово) | $[1,0,1,0]$ | $[1,0,1,0] + [1,0,1,0] = [2,0,2,0]$ |
| BoW (на документ) | $[1,0,1,0]$ | $[1,0,2,0]$ |

> В BoW видно, что слово *"бежит"* употреблено дважды — это отражено напрямую.  
> В One-Hot при суммировании теряется связь между счётчиком и конкретным словом (векторы просто складываются).



### 4. **Лучшее поведение в задачах машинного обучения**

- BoW векторы содержат **количественную информацию**, что позволяет алгоритмам (например, Naive Bayes, SVM) лучше различать документы.
- One-Hot векторы для документов (при суммировании) становятся **небинарными**, но при этом остаются **разрежёнными и несбалансированными**.



### 5. **Поддержка разреженных матриц**

Оба метода могут использовать разреженное хранение, но BoW **по умолчанию** работает с матрицами документов, что делает его **естественнее масштабируемым**.

Например, для корпуса из $N$ документов и словаря размера $V$:
- BoW: матрица $N \times V$ (разреженная),
- One-Hot: $N \times L \times V$, где $L$ — средняя длина документа (если хранить все слова отдельно).

> То есть BoW **в $L$ раз компактнее** при хранении коллекции документов.



## 2.2.6. Ограничения модели BoW

Несмотря на преимущества, BoW имеет свои недостатки:

| Ограничение | Пояснение |
|-----------|----------|
| **Игнорирование порядка слов** | Не различает *"собака кусает человека"* и *"человек кусает собаку"* |
| **Отсутствие семантики** | Не учитывает синонимы, морфологию, контекст |
| **Рост размерности** | При увеличении корпуса $V$ растёт, что увеличивает требования к памяти |
| **Чувствительность к шуму** | Частотные, но малозначимые слова (предлоги, союзы) могут доминировать |


## 2.2.7. Заключение

Модель **Bag-of-Words** является **существенным улучшением по сравнению с One-Hot Encoding** в контексте векторизации текстов. В отличие от One-Hot, который кодирует отдельные слова и игнорирует их частоту, BoW:
- позволяет представлять **целые документы**,
- учитывает **частоту слов**,
- формирует **информативные и интерпретируемые векторы**,
- эффективно масштабируется за счёт разреженных матриц.

Хотя BoW по-прежнему **не учитывает порядок слов и семантику**, он служит важным шагом на пути к более сложным моделям. Его простота, эффективность и хорошая производительность в задачах классификации делают BoW **одним из базовых инструментов в NLP**.

> В следующих главах мы рассмотрим методы, преодолевающие ограничения BoW, включая **TF-IDF**, **плотные эмбеддинги** и **контекстные модели**.




# 2.3. TF-IDF (Term Frequency – Inverse Document Frequency)

## Определение и назначение

**TF-IDF** (от англ. *Term Frequency – Inverse Document Frequency*) — это статистическая мера, используемая для оценки **важности слова** в документе относительно всей коллекции документов (корпуса). Метод является **улучшением модели Bag-of-Words (BoW)**, поскольку учитывает не только частоту слова в конкретном документе, но и его **редкость в корпусе в целом**.

Основная идея TF-IDF заключается в следующем:
- Слова, которые часто встречаются в одном документе, но редко — в других, являются **информативными** и должны иметь **высокий вес**.
- Слова, часто употребляемые во многих документах (например, "и", "в", "на", "это"), считаются **малозначимыми** (стоп-словами) и получают **пониженный вес**.

Таким образом, TF-IDF позволяет **автоматически подавлять шум** и выделять **ключевые термины**, характерные для конкретного документа.

## Математическая формула

TF-IDF вычисляется как произведение двух компонент:

1. **TF** (*Term Frequency*) — частота термина в документе,
2. **IDF** (*Inverse Document Frequency*) — мера редкости термина в корпусе.

Формально, вес слова $t$ в документе $d$ относительно корпуса $\mathcal{D}$ определяется как:

$$
\mathrm{TF\text{-}IDF}(t, d, \mathcal{D}) = \mathrm{TF}(t, d) \times \mathrm{IDF}(t, \mathcal{D})
$$


### 1. Term Frequency (TF)

**TF** — это мера того, насколько часто слово $t$ встречается в документе $d$. Существует несколько способов вычисления TF. Наиболее распространённые:

- **Абсолютная частота**:
  $$
  \mathrm{TF}(t, d) = \text{количество вхождений } t \text{ в } d
  $$

- **Нормализованная частота** (рекомендуется):
  $$
  \mathrm{TF}(t, d) = \frac{\text{количество вхождений } t \text{ в } d}{\text{общее число слов в } d}
  $$

Нормализация предотвращает предвзятость в пользу длинных документов, в которых слова могут встречаться чаще просто из-за объёма текста.



### 2. Inverse Document Frequency (IDF)

**IDF** — это мера того, насколько слово $t$ является **уникальным** или **редким** в корпусе $\mathcal{D}$. Чем реже слово встречается среди документов, тем выше его IDF.

Стандартная формула:

$$
\mathrm{IDF}(t, \mathcal{D}) = \log \left( \frac{N}{\mathrm{df}(t)} \right)
$$

где:
- $N$ — общее количество документов в корпусе,
- $\mathrm{df}(t)$ — число документов, содержащих слово $t$ (document frequency).

> **Примечание**:
> - Логарифм (обычно $\ln$ или $\log_{10}$) сглаживает значения и уменьшает влияние экстремально редких слов.
> - Чтобы избежать деления на ноль (если слово отсутствует в корпусе), часто используется сглаживание:
>   $$
>   \mathrm{IDF}(t, \mathcal{D}) = \log \left( \frac{N + 1}{\mathrm{df}(t) + 1} \right)
>   $$



## Аналитический пример вычисления TF-IDF

Рассмотрим корпус из трёх документов:

- $D_1$: *"Кот спит на диване"*
- $D_2$: *"Собака бежит по улице"*
- $D_3$: *"Кот и собака играют"*

Общее число документов: $N = 3$.

### Шаг 1: Вычисление Term Frequency (TF)

Словарь:  
{"кот", "спит", "на", "диване", "собака", "бежит", "по", "улице", "и", "играют"}

Используем **нормализованную частоту** (длина каждого документа — 4 слова).

| Слово | $D_1$ | $D_2$ | $D_3$ |
|------|-------|-------|-------|
| кот | $1/4 = 0.25$ | 0 | $0.25$ |
| спит | $0.25$ | 0 | 0 |
| на | $0.25$ | 0 | 0 |
| диване | $0.25$ | 0 | 0 |
| собака | 0 | $0.25$ | $0.25$ |
| бежит | 0 | $0.25$ | 0 |
| по | 0 | $0.25$ | 0 |
| улице | 0 | $0.25$ | 0 |
| и | 0 | 0 | $0.25$ |
| играют | 0 | 0 | $0.25$ |



### Шаг 2: Вычисление Inverse Document Frequency (IDF)

Используем натуральный логарифм: $\mathrm{IDF}(t) = \ln(3 / \mathrm{df}(t))$

| Слово | $\mathrm{df}(t)$ | $\mathrm{IDF}(t)$ |
|------|------------------|-------------------|
| кот | 2 | $\ln(3/2) \approx 0.405$ |
| спит | 1 | $\ln(3/1) \approx 1.098$ |
| на | 1 | $\approx 1.098$ |
| диване | 1 | $\approx 1.098$ |
| собака | 2 | $\approx 0.405$ |
| бежит | 1 | $\approx 1.098$ |
| по | 1 | $\approx 1.098$ |
| улице | 1 | $\approx 1.098$ |
| и | 1 | $\approx 1.098$ |
| играют | 1 | $\approx 1.098$ |



### Шаг 3: Вычисление TF-IDF

Перемножим значения TF и IDF.

**Документ $D_1$**:
- $\mathrm{TF\text{-}IDF}(\text{кот}, D_1) = 0.25 \times 0.405 = 0.101$
- $\mathrm{TF\text{-}IDF}(\text{спит}, D_1) = 0.25 \times 1.098 = 0.275$
- $\mathrm{TF\text{-}IDF}(\text{на}, D_1) = 0.25 \times 1.098 = 0.275$
- $\mathrm{TF\text{-}IDF}(\text{диване}, D_1) = 0.25 \times 1.098 = 0.275$

**Документ $D_2$**:
- $\mathrm{TF\text{-}IDF}(\text{собака}, D_2) = 0.25 \times 0.405 = 0.101$
- $\mathrm{TF\text{-}IDF}(\text{бежит}, D_2) = 0.25 \times 1.098 = 0.275$
- $\mathrm{TF\text{-}IDF}(\text{по}, D_2) = 0.25 \times 1.098 = 0.275$
- $\mathrm{TF\text{-}IDF}(\text{улице}, D_2) = 0.25 \times 1.098 = 0.275$

**Документ $D_3$**:
- $\mathrm{TF\text{-}IDF}(\text{кот}, D_3) = 0.25 \times 0.405 = 0.101$
- $\mathrm{TF\text{-}IDF}(\text{и}, D_3) = 0.25 \times 1.098 = 0.275$
- $\mathrm{TF\text{-}IDF}(\text{собака}, D_3) = 0.25 \times 0.405 = 0.101$
- $\mathrm{TF\text{-}IDF}(\text{играют}, D_3) = 0.25 \times 1.098 = 0.275$



## Анализ результатов

- Слова **"кот"** и **"собака"**, встречающиеся в двух документах, имеют **низкий IDF** ($\approx 0.405$) и, следовательно, **низкий TF-IDF** ($\approx 0.101$).
- Слова, встречающиеся **только в одном документе** (например, "спит", "диване", "и", "играют"), имеют **высокий IDF** ($\approx 1.098$) и **высокий TF-IDF** ($\approx 0.275$).

> Это демонстрирует, как TF-IDF **автоматически снижает вес общих слов** и **повышает вес уникальных терминов**, что делает документы более различимыми и информативными для задач классификации, поиска и кластеризации.



## Преимущества и ограничения TF-IDF

| Преимущество | Пояснение |
|-------------|----------|
| **Учёт информативности слов** | Редкие, но значимые слова получают более высокий вес |
| **Подавление стоп-слов** | Частотные, малозначимые слова (например, "на", "и") получают низкий IDF |
| **Интерпретируемость** | Высокие веса легко интерпретировать как ключевые слова документа |
| **Эффективность** | Поддерживает разреженные матрицы, что позволяет работать с большими корпусами |

| Ограничение | Пояснение |
|-----------|----------|
| **Игнорирование порядка слов** | Не различает *"собака кусает человека"* и *"человек кусает собаку"* |
| **Отсутствие семантики** | Не учитывает синонимы, морфологию, контекст |
| **Зависимость от корпуса** | IDF вычисляется на обучающем корпусе и не обновляется |
| **Разреженность векторов** | Векторы остаются разрежёнными, хотя и менее, чем в One-Hot Encoding |



## Заключение

TF-IDF является **ключевым улучшением модели BoW**, позволяя учитывать **информационную значимость слов** в контексте всего корпуса. Метод эффективно:
- снижает вес часто встречающихся, но малозначимых слов,
- повышает вес редких и информативных терминов,
- формирует более качественные векторные представления документов.

Хотя TF-IDF по-прежнему **не учитывает порядок слов и семантику**, он остаётся **широко применимым** в задачах:
- классификации текстов,
- информационного поиска,
- кластеризации документов.

В следующем разделе мы рассмотрим, как **расширение BoW и TF-IDF на N-граммы** позволяет частично учесть синтаксический контекст и улучшить качество векторизации.




# 2.2.3. Учёт униграмм и N-грамм

## Общая концепция

Модели **Bag-of-Words (BoW)** и **TF-IDF** по умолчанию основываются на **уникальных словах (униграммах)**, что приводит к полной потере информации о **порядке слов** и **синтаксических связях** между ними. Для частичного учёта контекста и улучшения качества векторного представления текста применяется расширение этих моделей с использованием **N-грамм** — непрерывных последовательностей из $N$ слов, извлекаемых из текста.

Пусть предложение состоит из $L$ слов: $w_1, w_2, \dots, w_L$. Тогда **N-грамма** порядка $N$ определяется как подпоследовательность:

$$
(w_i, w_{i+1}, \dots, w_{i+N-1}), \quad \text{где} \quad 1 \leq i \leq L - N + 1
$$

Количество N-грамм в предложении длины $L$ равно $L - N + 1$. Ниже рассмотрены наиболее употребительные типы N-грамм: **униграммы** ($N=1$), **биграммы** ($N=2$) и **триграммы** ($N=3$), с примерами реализации на основе библиотек `nltk` и `PySpark`.



### 1. Униграммы (1-граммы)

**Униграммы** — это отдельные слова, рассматриваемые как независимые признаки. Это базовый уровень представления текста, лежащий в основе классических моделей BoW и TF-IDF.

Множество униграмм:
$$
\{w_i \mid 1 \leq i \leq L\}
$$

#### Пример
Предложение: *"Я люблю NLP"*  
Длина $L = 3$  
Униграммы: `"Я"`, `"люблю"`, `"NLP"`

#### Реализация с использованием `nltk`



In [ ]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import nltk

# Загрузка ресурсов (один раз)
nltk.download('punkt')
nltk.download('punkt_tab')

# Текст
sentence = "Я люблю NLP"
tokens = word_tokenize(sentence.lower())  # Токенизация
L = len(tokens)  # Длина предложения

# Генерация униграмм (N=1)
unigrams = list(ngrams(tokens, 1))
print(f"Униграммы (L={L}): {unigrams}")
# Вывод: [('я',), ('люблю',), ('nlp',)]



> **Примечание**: Униграммы часто используются как базовый признаковый набор. Каждое слово становится отдельным признаком в векторе.



### 2. Биграммы (2-граммы)

**Биграммы** — это последовательные пары слов. Они позволяют учитывать **связи между соседними словами**, что критически важно для различения фраз с одинаковыми словами, но разным смыслом.

Множество биграмм:
$$
\{(w_i, w_{i+1}) \mid 1 \leq i \leq L-1\}
$$

#### Пример
Предложение: *"Я люблю NLP"*  
$L = 3$  
Биграммы: `"Я люблю"`, `"люблю NLP"`

#### Семантическая значимость
Биграммы помогают различать:
- *"горячая собака"* (еда),
- *"горячий пес"* (животное).

При использовании только униграмм эти фразы будут представлены одинаково, что приводит к потере семантики.

#### Реализация с использованием `nltk`



In [ ]:
from nltk.util import bigrams  # Алиас для ngrams(tokens, 2)

# Генерация биграмм
bigram_list = list(bigrams(tokens))
print(f"Биграммы (L={L}, число: {L-1}): {bigram_list}")



> **Альтернатива**: `ngrams(tokens, 2)` даёт тот же результат.



### 3. Триграммы (3-граммы)

**Триграммы** — это последовательности из трёх слов. Они захватывают **более широкий контекст**, что полезно для анализа сложных синтаксических конструкций и идиом.

Множество триграмм:
$$
\{(w_i, w_{i+1}, w_{i+2}) \mid 1 \leq i \leq L-2\}
$$

#### Пример
Предложение: *"Я люблю NLP"*  
$L = 3$  
Триграмма: `"Я люблю NLP"`

#### Семантическая значимость
Триграммы позволяют учитывать отрицания:
- *"не очень хорошо"* — триграмма `"не очень хорошо"` передаёт **отрицательную оценку**.
- Без триграмм модель может интерпретировать это как просто "хорошо".

#### Реализация с использованием `nltk`



In [ ]:
# Генерация триграмм
trigrams = list(ngrams(tokens, 3))
print(f"Триграммы (L={L}, число: {L-2}): {trigrams}")



### Расширение: N-граммы в распределённых системах (PySpark)

Для обработки больших корпусов текстов в промышленных приложениях часто используется **Apache Spark**, в частности модуль `pyspark.ml.feature.NGram`.

#### Пример с `pyspark.ml.feature.NGram`



In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Tokenizer

# Создание сессии Spark
spark = SparkSession.builder.appName("NGramExample").getOrCreate()

# Исходные данные
data = [(0, ["Я", "люблю", "NLP"]), (1, ["NLP", "очень", "интересна"])]
df = spark.createDataFrame(data, ["id", "words"])

# Генерация биграмм
ngram_transformer = NGram(n=2, inputCol="words", outputCol="ngrams")
ngram_df = ngram_transformer.transform(df)

# Просмотр результатов
ngram_df.select("ngrams").show(truncate=False)



> **Примечание**: `NGram` в PySpark автоматически генерирует N-граммы для каждого документа и поддерживает масштабирование на большие данные.



## Обобщение: Диапазон N-грамм

На практике редко используется только один тип N-грамм. Обычно задаётся **диапазон**, например:
$$
\text{ngram\_range} = (1, 2) \quad \text{или} \quad (1, 3)
$$

Это позволяет модели учитывать **униграммы, биграммы и триграммы** одновременно, что обеспечивает баланс между **семантической точностью** и **вычислительной эффективностью**.


## Преимущества и недостатки N-грамм

| Преимущество | Пояснение |
|-------------|----------|
| **Учёт локального контекста** | Модель видит, какие слова стоят рядом |
| **Различение фраз** | Позволяет отличать схожие по словам, но разные по смыслу конструкции |
| **Улучшение качества классификации** | Особенно полезно в задачах анализа тональности, детектирования спама и поиска |

| Недостаток | Пояснение |
|----------|----------|
| **Рост размерности словаря** | Каждая уникальная N-грамма — новый признак. Словарь растёт экспоненциально |
| **Разреженность векторов** | Многие N-граммы встречаются редко, что приводит к разрежённым векторам |
| **Ограниченный контекст** | Учитываются только локальные связи, но не глобальный смысл предложения |


## Практическая реализация на Python

Ниже приведён **подробный и прокомментированный код**, демонстрирующий, как использовать TF-IDF с разными типами N-грамм.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример корпуса документов
documents = [
    "Кот спит на диване",
    "Собака бежит по улице",
    "Кот и собака играют",
    "Собака спит на коврике"
]

print("=== 1. УНИГРАММЫ (1-граммы) ===")
# Создаём векторизатор для униграмм: ngram_range=(1,1)
tfidf_unigram = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
# Обучаем и преобразуем
X_unigram = tfidf_unigram.fit_transform(documents)
# Получаем словарь и векторы
vocab_unigram = tfidf_unigram.get_feature_names_out()
print(f"Словарь униграмм: {list(vocab_unigram)}")
print("TF-IDF матрица (каждая строка — документ):")
print(X_unigram.toarray())
print()

print("=== 2. БИГРАММЫ (2-граммы) ===")
# Создаём векторизатор для биграмм: ngram_range=(2,2)
tfidf_bigram = TfidfVectorizer(ngram_range=(2, 2), lowercase=True)
X_bigram = tfidf_bigram.fit_transform(documents)
vocab_bigram = tfidf_bigram.get_feature_names_out()
print(f"Словарь биграмм: {list(vocab_bigram)}")
print("TF-IDF матрица:")
print(X_bigram.toarray())
print()

print("=== 3. УНИГРАММЫ + БИГРАММЫ (диапазон 1–2) ===")
# Создаём векторизатор, учитывающий и 1-граммы, и 2-граммы
tfidf_12 = TfidfVectorizer(ngram_range=(1, 2), lowercase=True)
X_12 = tfidf_12.fit_transform(documents)
vocab_12 = tfidf_12.get_feature_names_out()
print(f"Словарь (униграммы + биграммы): {list(vocab_12)}")
print("TF-IDF матрица:")
print(X_12.toarray())
print()

print("=== 4. ПРИМЕР: ВЕКТОРИЗАЦИЯ НОВОГО ДОКУМЕНТА ===")
# Новый документ (не в обучающем корпусе)
new_doc = ["Кот спит"]
# Преобразуем с помощью обученного векторизатора
new_vec = tfidf_12.transform(new_doc)
print(f"Новый документ: '{new_doc[0]}'")
print(f"Его TF-IDF вектор (размерность: {new_vec.shape[1]}):")
print(new_vec.toarray())
print("Ненулевые признаки:")
# Находим индексы ненулевых элементов
nonzero_idx = new_vec.nonzero()[1]
for idx in nonzero_idx:
    feature = vocab_12[idx]
    weight = new_vec[0, idx]
    print(f"  '{feature}': {weight:.4f}")


## Анализ результата

Для документа `"Кот спит"` векторизатор с `ngram_range=(1,2)` создаст признаки:
- Униграммы: `"кот"`, `"спит"`
- Биграммы: `"кот спит"`

Если биграмма `"кот спит"` встречалась в обучающем корпусе, она получит **собственный вес**, отражающий её информативность. Это позволяет модели учитывать **фразы как единые смысловые единицы**.





## Заключение

Расширение моделей BoW и TF-IDF с помощью **N-грамм** позволяет частично преодолеть их ключевой недостаток — **игнорирование порядка слов**. Использование униграмм, биграмм и триграмм обеспечивает разный уровень контекстуального охвата:

| Тип | Число N-грамм | Контекст |
|-----|----------------|----------|
| Униграммы | $L$ | Отдельные слова |
| Биграммы | $L-1$ | Пары слов |
| Триграммы | $L-2$ | Тройки слов |



Хотя это приводит к **росту размерности** и **разрежённости**, на практике использование диапазона `(1,2)` или `(1,3)` часто даёт **значительный прирост качества** при разумных вычислительных затратах.

Для научных и образовательных задач рекомендуется использовать `nltk.util.ngrams`, а для обработки больших данных — `pyspark.ml.feature.NGram`. В следующих главах мы рассмотрим **плотные векторные эмбеддинги**, которые решают проблему контекста более глубоко.



# 3. Word2Vec: Подробное описание

## Введение

**Word2Vec** — это семейство моделей машинного обучения, разработанных Томашем Миколовым и его коллегами в Google в 2013 году, предназначенных для эффективного обучения **плотных векторных представлений слов**, известных как **словные эмбеддинги (word embeddings)**. Эти эмбеддинги представляют слова в непрерывном многомерном векторном пространстве, где **семантическая близость** слов отражается через **геометрическую близость** их векторов.

Основное преимущество Word2Vec заключается в способности модели улавливать **семантические и синтаксические отношения** между словами. Например, векторы слов *"король"* и *"королева"* оказываются близки друг к другу, а арифметические операции вида:
$$
\mathrm{vec}(\text{король}) - \mathrm{vec}(\text{мужчина}) + \mathrm{vec}(\text{женщина}) \approx \mathrm{vec}(\text{королева})
$$
демонстрируют, что модель способна усваивать абстрактные лингвистические закономерности.



## Теоретическая основа: Гипотеза распределения

В основе Word2Vec лежит **гипотеза распределения (Distributional Hypothesis)**, сформулированная Дж. Р. Фирсом:  
> *"Слова, которые появляются в похожих контекстах, имеют схожие значения."*

Эта гипотеза лежит в основе всех современных методов обучения векторных представлений слов. Word2Vec реализует её, обучаясь предсказывать слова на основе их окружения (контекста), что позволяет модели неявно усваивать лексико-семантические паттерны.



## Архитектура модели

Word2Vec не является глубокой нейронной сетью в классическом понимании. Это **двухслойная нейросеть прямого распространения (shallow neural network)** с тремя основными слоями:
1. **Входной слой** — кодирует входное слово (или контекст) в виде one-hot вектора.
2. **Проекционный (скрытый) слой** — содержит $d$ нейронов, где $d$ — размерность эмбеддингов. Веса этого слоя и есть искомые векторные представления слов.
3. **Выходной слой** — вычисляет вероятности всех слов в словаре для задачи классификации.

Модель обучается решению **вспомогательной задачи (proxy task)** — предсказанию слова по контексту или наоборот. В процессе обучения веса проекционного слоя настраиваются так, чтобы максимизировать вероятность правильного предсказания. После обучения эти веса используются как **словные эмбеддинги**.



## Две основные архитектуры

Word2Vec предлагает две архитектуры, различающиеся направлением предсказания:

### 1. CBOW (Continuous Bag-of-Words)  
Предсказывает целевое (центральное) слово на основе его контекстных слов.

### 2. Skip-gram  
Предсказывает контекстные слова на основе целевого (центрального) слова.


## Математические основы модели Skip-gram

### 1. Введение в векторные представления слов (Word Embeddings)

В области обработки естественного языка (NLP) традиционные методы представления слов, такие как унитарное (one-hot) кодирование, страдают от проблемы *"проклятия размерности"* и неспособности улавливать семантические или синтаксические отношения между словами. Каждое слово представляется как независимая сущность, что не позволяет моделировать сходство между словами, например, между *"король"* и *"королева"* или *"быстрый"* и *"скорый"*.

Векторные представления слов, или **word embeddings**, решают эту проблему, отображая слова из дискретного пространства в непрерывное векторное пространство низкой размерности. В этом пространстве слова с похожим значением или контекстом располагаются близко друг к другу. Такие представления являются основой для многих современных NLP-задач, включая машинный перевод, анализ настроений и вопросно-ответные системы.

Модель **Skip-gram**, разработанная Томашем Миколовым и его коллегами в Google, является одной из наиболее популярных и эффективных архитектур для обучения векторных представлений слов. Она относится к семейству моделей **Word2Vec**.

#### 1.1. Основная идея Skip-gram

Основная идея модели Skip-gram заключается в предсказании контекстных слов, окружающих данное целевое слово. Если модель может успешно предсказывать контекст, значит, она «понимает» что-то о значении целевого слова, и это «понимание» кодируется в его векторном представлении.

Пусть у нас есть предложение: *"Кот сидит на коврике"*. Если *"сидит"* является целевым словом, то *"Кот"*, *"на"*, *"коврике"* могут быть его контекстными словами в пределах определённого окна. Модель Skip-gram пытается максимизировать вероятность наблюдения контекстных слов $w_c$ при данном целевом слове $w_t$:
$$
P(w_c \mid w_t)
$$
Это отличается от модели **CBOW** (Continuous Bag of Words), которая предсказывает целевое слово на основе его контекста.



### 2. Архитектура модели Skip-gram

Модель Skip-gram представляет собой простую двухслойную нейронную сеть (без нелинейности в скрытом слое), которая обучается на большом корпусе текста.

#### 2.1. Слои модели

1. **Входной слой (Input Layer)**: Представляет целевое слово $w_t$ в виде унитарного (one-hot) вектора. Размерность этого вектора равна размеру словаря $V$.
2. **Скрытый слой (Hidden Layer)**: Этот слой не имеет функции активации (или имеет линейную функцию активации). Количество нейронов в этом слое равно желаемой размерности векторного представления слова $N$. Выход этого слоя является векторным представлением (эмбеддингом) входного слова.
3. **Выходной слой (Output Layer)**: Имеет $V$ нейронов, по одному для каждого слова в словаре. Каждый нейрон выдает оценку (score) того, насколько вероятно, что соответствующее слово является контекстным словом для данного целевого слова. Затем к этим оценкам применяется функция **Softmax** для получения вероятностей.

#### 2.2. Весовые матрицы

В модели Skip-gram есть две основные весовые матрицы:

- **Матрица весов "вход–скрытый слой" ($W_{\text{in}}$)**: Эта матрица имеет размерность $V \times N$. Каждая строка этой матрицы представляет собой $N$-мерный вектор, который является эмбеддингом соответствующего слова, когда оно выступает в роли входного (целевого) слова. Мы будем называть эти векторы **входными эмбеддингами** и обозначать их как $\mathbf{v}_w$.

- **Матрица весов "скрытый слой–выход" ($W_{\text{out}}$)**: Эта матрица имеет размерность $N \times V$. Каждый столбец этой матрицы представляет собой $N$-мерный вектор, который является эмбеддингом соответствующего слова, когда оно выступает в роли контекстного слова. Мы будем называть эти векторы **выходными эмбеддингами** и обозначать их как $\mathbf{u}_w$.

> **Важно**: Для каждого слова $w$ в словаре существует два векторных представления: $\mathbf{v}_w$ (когда $w$ — целевое слово) и $\mathbf{u}_w$ (когда $w$ — контекстное слово). В конце обучения обычно используется $\mathbf{v}_w$ в качестве окончательного эмбеддинга слова.



### 3. Алгоритм прямого прохода (Forward Pass)

Прямой проход — это процесс вычисления выходных вероятностей модели на основе заданного входного (целевого) слова.

Пусть $w_t$ — целевое слово, представленное one-hot вектором $\mathbf{x}_t$ размерности $V$.

#### 3.1. Вычисление скрытого слоя

One-hot вектор $\mathbf{x}_t$ имеет единицу на позиции, соответствующей $w_t$, и нули в остальных позициях. При умножении на матрицу $W_{\text{in}}$ ($V \times N$) результатом является $N$-мерный вектор, соответствующий строке матрицы $W_{\text{in}}$, связанной со словом $w_t$. Этот вектор и есть входной эмбеддинг слова $w_t$, обозначаемый как $\mathbf{v}_{w_t}$:
$$
\mathbf{h} = \mathbf{x}_t^\top W_{\text{in}} = \mathbf{v}_{w_t}
$$
Здесь $\mathbf{h}$ — вектор скрытого слоя, который фактически является эмбеддингом целевого слова $w_t$.

#### 3.2. Вычисление оценок для выходного слоя

Вектор скрытого слоя $\mathbf{h}$ умножается на матрицу $W_{\text{out}}$ ($N \times V$). Результат — $V$-мерный вектор оценок $\mathbf{z}$, где каждый элемент $z_j$ представляет собой оценку для $j$-го слова в словаре:
$$
\mathbf{z} = \mathbf{h}^\top W_{\text{out}} = \mathbf{v}_{w_t}^\top W_{\text{out}}
$$
Каждый элемент $z_j$ может быть интерпретирован как скалярное произведение входного эмбеддинга целевого слова $\mathbf{v}_{w_t}$ и выходного эмбеддинга контекстного слова $\mathbf{u}_{w_j}$:
$$
z_j = \mathbf{v}_{w_t}^\top \mathbf{u}_{w_j}
$$
Высокое значение $z_j$ означает, что слова $w_t$ и $w_j$ часто встречаются вместе в контексте.

#### 3.3. Применение функции Softmax

Для преобразования оценок $\mathbf{z}$ в вероятности $P(w_j \mid w_t)$, которые суммируются к 1, используется функция **Softmax**:
$$
P(w_j \mid w_t) = \frac{\exp(z_j)}{\sum_{k=1}^{V} \exp(z_k)} = \frac{\exp(\mathbf{v}_{w_t}^\top \mathbf{u}_{w_j})}{\sum_{k=1}^{V} \exp(\mathbf{v}_{w_t}^\top \mathbf{u}_{w_k})}
$$
Это и есть предсказанная вероятность того, что слово $w_j$ является контекстным словом для $w_t$.



### 4. Функция потерь (Loss Function)

Цель обучения модели Skip-gram — максимизировать вероятность наблюдения фактических контекстных слов для каждого целевого слова. Это эквивалентно минимизации отрицательного логарифма этой вероятности.

Для каждой пары (целевое слово $w_t$, контекстное слово $w_c$) в обучающем наборе функция потерь (кросс-энтропия) определяется как:
$$
\mathcal{L}(w_t, w_c) = -\log P(w_c \mid w_t)
$$

Для всего обучающего набора, состоящего из пар $(w_t, w_c)$, извлечённых из корпуса текста, общая функция потерь, которую мы хотим минимизировать, имеет вид:
$$
\mathcal{L} = -\sum_{(w_t, w_c) \in D} \log P(w_c \mid w_t)
$$
где $D$ — множество всех пар (целевое слово, контекстное слово) из обучающего корпуса.



> **Примечание**: В реальных реализациях прямое применение Softmax вычислительно затратно из-за необходимости суммирования по всему словарю ($V$). Поэтому на практике часто используются приближённые методы, такие как **Negative Sampling** или **Hierarchical Softmax**, которые значительно ускоряют обучение.


# 5. Алгоритм обратного распространения ошибки (Backpropagation)

**Обратное распространение ошибки (Backpropagation)** — это ключевой алгоритм в обучении нейронных сетей, позволяющий эффективно вычислять градиенты функции потерь по отношению к весам модели. Эти градиенты используются для обновления весов с целью минимизации потерь с помощью методов оптимизации, таких как стохастический градиентный спуск (SGD).

Для модели **Skip-gram** мы хотим вычислить градиенты по двум весовым матрицам:
- $\frac{\partial \mathcal{L}}{\partial W_{\text{in}}}$ — по входной матрице (входные эмбеддинги $\mathbf{v}_w$),
- $\frac{\partial \mathcal{L}}{\partial W_{\text{out}}}$ — по выходной матрице (выходные эмбеддинги $\mathbf{u}_w$).

Рассмотрим процесс для одной обучающей пары $(w_t, w_c)$, где $w_t$ — целевое слово, $w_c$ — контекстное слово.



### 5.1. Градиент по выходному слою

Начнём с вычисления градиента функции потерь по отношению к вектору оценок $z_j$, полученных на выходном слое.

Функция потерь для одной пары:
$$
\mathcal{L} = -\log P(w_c \mid w_t) = -\log \left( \frac{\exp(z_c)}{\sum_{k=1}^{V} \exp(z_k)} \right)
$$
где $z_c = \mathbf{v}_{w_t}^\top \mathbf{u}_{w_c}$ — оценка для истинного контекстного слова $w_c$.

Производная потерь по оценке $z_j$ известна из свойств **Softmax + кросс-энтропия**:
$$
\frac{\partial \mathcal{L}}{\partial z_j} = P(w_j \mid w_t) - y_j
$$
где:
- $P(w_j \mid w_t)$ — предсказанная вероятность слова $w_j$,
- $y_j = \begin{cases} 1, & \text{если } j = c \\ 0, & \text{иначе} \end{cases}$ — истинная метка (one-hot вектор).

Обозначим эту разницу как **ошибку на выходе**:
$$
e_j = P(w_j \mid w_t) - y_j
$$
Вектор ошибок $\mathbf{e} = [e_1, e_2, \dots, e_V]^\top$ показывает, насколько модель ошиблась при предсказании каждого слова в словаре.

#### Обновление выходных эмбеддингов

Напомним: $z_j = \mathbf{h}^\top \mathbf{u}_{w_j} = \mathbf{v}_{w_t}^\top \mathbf{u}_{w_j}$, где $\mathbf{u}_{w_j}$ — $j$-й столбец матрицы $W_{\text{out}}$.

Тогда градиент потерь по выходному эмбеддингу слова $w_j$:
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{u}_{w_j}} = \frac{\partial \mathcal{L}}{\partial z_j} \cdot \frac{\partial z_j}{\partial \mathbf{u}_{w_j}} = e_j \cdot \mathbf{h} = e_j \cdot \mathbf{v}_{w_t}
$$

Обновление весов (с шагом обучения $\eta$):
$$
\mathbf{u}_{w_j}^{\text{new}} = \mathbf{u}_{w_j}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{u}_{w_j}} = \mathbf{u}_{w_j}^{\text{old}} - \eta \cdot e_j \cdot \mathbf{v}_{w_t}
$$

> ⚠️ **Важно**: Это обновление применяется ко **всем** словам в словаре, что вычислительно затратно. Именно поэтому на практике используется **Negative Sampling** (см. далее).



### 5.2. Градиент по скрытому слою

Теперь вычислим градиент функции потерь по вектору скрытого слоя $\mathbf{h} = \mathbf{v}_{w_t}$, чтобы передать ошибку назад к входному слою.

По цепному правилу:
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{h}} = \sum_{j=1}^{V} \frac{\partial \mathcal{L}}{\partial z_j} \cdot \frac{\partial z_j}{\partial \mathbf{h}} = \sum_{j=1}^{V} e_j \cdot \mathbf{u}_{w_j}
$$

Этот вектор:
$$
\delta_{\mathbf{h}} = \frac{\partial \mathcal{L}}{\partial \mathbf{h}} = \sum_{j=1}^{V} e_j \cdot \mathbf{u}_{w_j}
$$
представляет собой **взвешенную сумму выходных эмбеддингов**, где веса — ошибки $e_j$. Он показывает, как нужно изменить эмбеддинг целевого слова $w_t$, чтобы уменьшить потери.


### 5.3. Градиент по входному слою (обновление $W_{\text{in}}$)

Напомним: $\mathbf{h} = \mathbf{x}_t^\top W_{\text{in}}$, где $\mathbf{x}_t$ — one-hot вектор для $w_t$. Это означает, что $\mathbf{h}$ — это просто $t$-я строка матрицы $W_{\text{in}}$, т.е. $\mathbf{v}_{w_t}$.

Следовательно, градиент по $W_{\text{in}}$ затрагивает **только одну строку** — соответствующую $w_t$:
$$
\frac{\partial \mathcal{L}}{\partial W_{\text{in}}} = \mathbf{x}_t \cdot \left( \frac{\partial \mathcal{L}}{\partial \mathbf{h}} \right)^\top
$$
Поскольку $\mathbf{x}_t$ — one-hot вектор, результат — матрица, у которой только $t$-я строка ненулевая и равна $\left( \frac{\partial \mathcal{L}}{\partial \mathbf{h}} \right)^\top$.

Таким образом, обновление входного эмбеддинга:
$$
\mathbf{v}_{w_t}^{\text{new}} = \mathbf{v}_{w_t}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{h}} = \mathbf{v}_{w_t}^{\text{old}} - \eta \cdot \sum_{j=1}^{V} e_j \cdot \mathbf{u}_{w_j}
$$



### 5.4. Итоговый алгоритм обучения (с полным Softmax)

Для каждой пары $(w_t, w_c)$ в контекстном окне:
1. **Прямой проход**:
   - Получить $\mathbf{v}_{w_t}$ из $W_{\text{in}}$.
   - Вычислить $z_j = \mathbf{v}_{w_t}^\top \mathbf{u}_{w_j}$ для всех $j$.
   - Применить Softmax: $P(w_j \mid w_t) = \frac{\exp(z_j)}{\sum_k \exp(z_k)}$.
2. **Вычисление ошибок**:
   - $e_j = P(w_j \mid w_t) - y_j$, где $y_j = 1$ только если $j = c$.
3. **Обратное распространение**:
   - Обновить все выходные эмбеддинги:  
     $\mathbf{u}_{w_j} \leftarrow \mathbf{u}_{w_j} - \eta \cdot e_j \cdot \mathbf{v}_{w_t}$.
   - Вычислить $\delta_{\mathbf{h}} = \sum_j e_j \cdot \mathbf{u}_{w_j}$.
   - Обновить входной эмбеддинг:  
     $\mathbf{v}_{w_t} \leftarrow \mathbf{v}_{w_t} - \eta \cdot \delta_{\mathbf{h}}$.

> ❌ **Проблема**: Шаг 3 требует обновления $V$ векторов (по размеру словаря), что крайне неэффективно при $V \sim 10^5{-}10^6$.



# 6. Оптимизация: Negative Sampling (Отрицательное сэмплирование)

Для ускорения обучения вместо полного Softmax используется **Negative Sampling (NS)** — приближённый метод, заменяющий многоклассовую задачу на серию бинарных классификаций.

## 6.1. Принцип работы

Для каждой **положительной пары** $(w_t, w_c)$, которая реально встречается в тексте, модель учится:
- **Подтверждать** эту пару («это хороший контекст»),
- **Опровергать** $k$ случайно выбранных **отрицательных пар** $(w_t, w_{\text{neg},i})$, где $w_{\text{neg},i}$ — слова, **не** входящие в контекст $w_t$.

Цель: научить модель отличать "настоящие" контексты от "случайных".



## 6.2. Функция потерь с Negative Sampling

Функция потерь строится на основе **логистической регрессии** (сигмоиды):
$$
\mathcal{L}_{\text{NS}} = -\log \sigma(\mathbf{v}_{w_t}^\top \mathbf{u}_{w_c}) - \sum_{i=1}^{k} \log \sigma(-\mathbf{v}_{w_t}^\top \mathbf{u}_{w_{\text{neg},i}})
$$
где:
- $\sigma(x) = \frac{1}{1 + \exp(-x)}$ — сигмоида,
- $k$ — количество отрицательных примеров (обычно 5–20).

- Первый член максимизирует вероятность **наличия** связи $w_t$–$w_c$,
- Второй член максимизирует вероятность **отсутствия** связи $w_t$–$w_{\text{neg},i}$.



## 6.3. Обратное распространение с Negative Sampling

Градиенты теперь вычисляются только для **малого числа векторов** — $w_t$, $w_c$ и $k$ отрицательных слов.

#### Для положительного примера $(w_t, w_c)$:

Пусть $s = \mathbf{v}_{w_t}^\top \mathbf{u}_{w_c}$. Тогда:
$$
\frac{\partial \mathcal{L}_{\text{NS}}}{\partial \mathbf{v}_{w_t}} = (\sigma(s) - 1) \cdot \mathbf{u}_{w_c}, \quad
\frac{\partial \mathcal{L}_{\text{NS}}}{\partial \mathbf{u}_{w_c}} = (\sigma(s) - 1) \cdot \mathbf{v}_{w_t}
$$

#### Для каждого отрицательного примера $(w_t, w_{\text{neg},i})$:

Пусть $s_i = \mathbf{v}_{w_t}^\top \mathbf{u}_{w_{\text{neg},i}}$. Тогда:
$$
\frac{\partial \mathcal{L}_{\text{NS}}}{\partial \mathbf{v}_{w_t}} \mathrel{+}= \sigma(s_i) \cdot \mathbf{u}_{w_{\text{neg},i}}, \quad
\frac{\partial \mathcal{L}_{\text{NS}}}{\partial \mathbf{u}_{w_{\text{neg},i}}} = \sigma(s_i) \cdot \mathbf{v}_{w_t}
$$

> 🔁 **Обновляются только**:
> - $\mathbf{v}_{w_t}$ (входной эмбеддинг целевого слова),
> - $\mathbf{u}_{w_c}$ (выходной эмбеддинг положительного слова),
> - $\mathbf{u}_{w_{\text{neg},i}}$ (выходные эмбеддинги $k$ отрицательных слов).

✅ **Преимущества**:
- Вычислительная сложность снизилась с $O(V)$ до $O(k)$.
- Обучение становится на порядки быстрее.
- Сохраняется качество эмбеддингов.


# 7. Заключение

Модель **Skip-gram** является мощным и эффективным инструментом для обучения векторных представлений слов. Её математическая основа — комбинация простой архитектуры, гипотезы распределения и градиентного обучения — позволяет модели улавливать как семантические, так и синтаксические закономерности.

Ключевые компоненты:
- **Прямой проход** формирует вероятности контекстных слов.
- **Обратное распространение** позволяет эффективно обновлять веса.
- **Negative Sampling** делает обучение масштабируемым, заменяя дорогой Softmax на быстрые бинарные классификации.

Понимание этих механизмов критически важно не только для использования Word2Vec, но и для освоения более сложных моделей NLP, таких как **GloVe**, **FastText**, и современных трансформеров, где идеи векторных представлений и контекстного обучения развиваются дальше.




# 8. Аналитический пример: Пошаговое вычисление Skip-gram

В этом разделе мы проведём **пошаговое вычисление** прямого и обратного прохода модели **Skip-gram** на конкретном примере. Все вычисления будут выполнены вручную, чтобы продемонстрировать, как модель обучается на одной обучающей паре.



## 1. Исходные данные и инициализация

**Предложение для обучения**:  
> "кот сидит на коврике"

**Словарь** (размер $V = 4$):  
$$
\text{слово} \mapsto \text{индекс:} \quad
\begin{cases}
\text{кот} &\mapsto 0 \\
\text{сидит} &\mapsto 1 \\
\text{на} &\mapsto 2 \\
\text{коврике} &\mapsto 3
\end{cases}
$$

**Размерность эмбеддингов** $N = 2$ — каждое слово представляется 2-мерным вектором.

**One-hot векторы**:
$$
\mathbf{x}_{\text{кот}} = [1, 0, 0, 0]^\top, \quad
\mathbf{x}_{\text{сидит}} = [0, 1, 0, 0]^\top, \\
\mathbf{x}_{\text{на}} = [0, 0, 1, 0]^\top, \quad
\mathbf{x}_{\text{коврике}} = [0, 0, 0, 1]^\top
$$

**Скорость обучения**: $\eta = 0.01$



### Инициализация весовых матриц

Матрицы инициализируются случайными малыми значениями.

#### Матрица $W_{\text{in}}$ (входные эмбеддинги $\mathbf{v}_w$)

Размерность: $V \times N = 4 \times 2$  
Каждая строка — это входной эмбеддинг слова:

$$
W_{\text{in}} =
\begin{bmatrix}
\mathbf{v}_{\text{кот}} \\
\mathbf{v}_{\text{сидит}} \\
\mathbf{v}_{\text{на}} \\
\mathbf{v}_{\text{коврике}}
\end{bmatrix}
=
\begin{bmatrix}
0.1 & 0.3 \\
0.5 & 0.7 \\
0.2 & 0.4 \\
0.6 & 0.8
\end{bmatrix}
$$

То есть:
- $\mathbf{v}_{\text{кот}} = [0.1, 0.3]$
- $\mathbf{v}_{\text{сидит}} = [0.5, 0.7]$
- $\mathbf{v}_{\text{на}} = [0.2, 0.4]$
- $\mathbf{v}_{\text{коврике}} = [0.6, 0.8]$

> ⚠️ **Ошибка в оригинале**: в тексте было указано $\mathbf{v}_{\text{сидит}} = [0.3, 0.4]$, но это не соответствует второй строке матрицы. Исправлено.

#### Матрица $W_{\text{out}}$ (выходные эмбеддинги $\mathbf{u}_w$)

Размерность: $N \times V = 2 \times 4$  
Каждый столбец — это выходной эмбеддинг слова:

$$
W_{\text{out}} =
\begin{bmatrix}
0.05 & 0.15 & 0.25 & 0.35 \\
0.10 & 0.20 & 0.30 & 0.40
\end{bmatrix}
$$

То есть:
- $\mathbf{u}_{\text{кот}} = [0.05, 0.10]^\top$
- $\mathbf{u}_{\text{сидит}} = [0.15, 0.20]^\top$
- $\mathbf{u}_{\text{на}} = [0.25, 0.30]^\top$
- $\mathbf{u}_{\text{коврике}} = [0.35, 0.40]^\top$



## 2. Выбор обучающей пары

Выберем:
- **Целевое слово** $w_t = \text{«сидит»}$ (индекс 1)
- **Контекстное слово** $w_c = \text{«кот»}$ (индекс 0)

(Предположим, что контекстное окно размером 1 слева и справа.)



## 3. Прямой проход (Forward Pass)

### 3.1. Вычисление скрытого слоя $\mathbf{h}$

Скрытый слой — это эмбеддинг целевого слова:
$$
\mathbf{h} = \mathbf{x}_{\text{сидит}}^\top W_{\text{in}} = \text{вторая строка } W_{\text{in}}
$$
$$
\mathbf{h} = \mathbf{v}_{\text{сидит}} = [0.5, 0.7]
$$



### 3.2. Вычисление оценок $z_j$ на выходном слое

$$
\mathbf{z} = \mathbf{h}^\top W_{\text{out}} = [0.5, 0.7]
\begin{bmatrix}
0.05 & 0.15 & 0.25 & 0.35 \\
0.10 & 0.20 & 0.30 & 0.40
\end{bmatrix}
$$

Вычислим скалярные произведения:
- $z_{\text{кот}} = \mathbf{v}_{\text{сидит}}^\top \mathbf{u}_{\text{кот}} = 0.5 \cdot 0.05 + 0.7 \cdot 0.10 = 0.025 + 0.070 = 0.095$
- $z_{\text{сидит}} = 0.5 \cdot 0.15 + 0.7 \cdot 0.20 = 0.075 + 0.140 = 0.215$
- $z_{\text{на}} = 0.5 \cdot 0.25 + 0.7 \cdot 0.30 = 0.125 + 0.210 = 0.335$
- $z_{\text{коврике}} = 0.5 \cdot 0.35 + 0.7 \cdot 0.40 = 0.175 + 0.280 = 0.455$

Итак:
$$
\mathbf{z} = [0.095, 0.215, 0.335, 0.455]
$$



### 3.3. Применение Softmax

$$
P(w_j \mid w_t) = \frac{\exp(z_j)}{\sum_{k=0}^{3} \exp(z_k)}
$$

Вычислим экспоненты:
- $\exp(0.095) \approx 1.0996$
- $\exp(0.215) \approx 1.2399$
- $\exp(0.335) \approx 1.3977$
- $\exp(0.455) \approx 1.5758$

Сумма:  
$$
\sum_k \exp(z_k) = 1.0996 + 1.2399 + 1.3977 + 1.5758 = 5.3130
$$

Теперь вероятности:
- $P(\text{кот} \mid \text{сидит}) = \frac{1.0996}{5.3130} \approx 0.2069$
- $P(\text{сидит} \mid \text{сидит}) = \frac{1.2399}{5.3130} \approx 0.2334$
- $P(\text{на} \mid \text{сидит}) = \frac{1.3977}{5.3130} \approx 0.2631$
- $P(\text{коврике} \mid \text{сидит}) = \frac{1.5758}{5.3130} \approx 0.2966$

Проверка: $0.2069 + 0.2334 + 0.2631 + 0.2966 = 1.0000$ ✅


## 4. Функция потерь

Истинное контекстное слово: $w_c = \text{«кот»}$ → $y = [1, 0, 0, 0]^\top$

Функция потерь (кросс-энтропия):
$$
\mathcal{L} = -\log P(\text{кот} \mid \text{сидит}) = -\log(0.2069) \approx -(-1.575) = 1.575
$$

> Цель: уменьшить $\mathcal{L}$, увеличив $P(\text{кот} \mid \text{сидит})$.



## 5. Обратное распространение ошибки (Backpropagation)

### 5.1. Вычисление ошибок $e_j$

$$
e_j = P(w_j \mid w_t) - y_j
$$

- $e_{\text{кот}} = 0.2069 - 1 = -0.7931$
- $e_{\text{сидит}} = 0.2334 - 0 = 0.2334$
- $e_{\text{на}} = 0.2631 - 0 = 0.2631$
- $e_{\text{коврике}} = 0.2966 - 0 = 0.2966$

Вектор ошибок:  
$$
\mathbf{e} = [-0.7931, 0.2334, 0.2631, 0.2966]^\top
$$



### 5.2. Обновление выходных эмбеддингов $\mathbf{u}_{w_j}$

Градиент:
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{u}_{w_j}} = e_j \cdot \mathbf{h} = e_j \cdot [0.5, 0.7]
$$

Обновление:
$$
\mathbf{u}_{w_j}^{\text{new}} = \mathbf{u}_{w_j}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{u}_{w_j}}
$$

#### Для $\mathbf{u}_{\text{кот}}$:
- Градиент: $-0.7931 \cdot [0.5, 0.7] = [-0.39655, -0.55517]$
- Обновление:  
  $$
  \mathbf{u}_{\text{кот}}^{\text{new}} = [0.05, 0.10] - 0.01 \cdot [-0.39655, -0.55517] = [0.05, 0.10] + [0.0039655, 0.0055517]
  $$
  $$
  \mathbf{u}_{\text{кот}}^{\text{new}} \approx [0.05397, 0.10555]
  $$

#### Для $\mathbf{u}_{\text{сидит}}$:
- Градиент: $0.2334 \cdot [0.5, 0.7] = [0.1167, 0.16338]$
- Обновление:  
  $$
  \mathbf{u}_{\text{сидит}}^{\text{new}} = [0.15, 0.20] - 0.01 \cdot [0.1167, 0.16338] = [0.15, 0.20] - [0.001167, 0.0016338]
  $$
  $$
  \mathbf{u}_{\text{сидит}}^{\text{new}} \approx [0.14883, 0.19837]
  $$

(Аналогично обновляются $\mathbf{u}_{\text{на}}$ и $\mathbf{u}_{\text{коврике}}$, но мы их опустим для краткости.)



### 5.3. Вычисление градиента по скрытому слою $\frac{\partial \mathcal{L}}{\partial \mathbf{h}}$

$$
\frac{\partial \mathcal{L}}{\partial \mathbf{h}} = \sum_{j=0}^{3} e_j \cdot \mathbf{u}_{w_j}^{\text{old}}
$$

Вычислим:
- $e_{\text{кот}} \cdot \mathbf{u}_{\text{кот}} = -0.7931 \cdot [0.05, 0.10] = [-0.039655, -0.07931]$
- $e_{\text{сидит}} \cdot \mathbf{u}_{\text{сидит}} = 0.2334 \cdot [0.15, 0.20] = [0.03501, 0.04668]$
- $e_{\text{на}} \cdot \mathbf{u}_{\text{на}} = 0.2631 \cdot [0.25, 0.30] = [0.065775, 0.07893]$
- $e_{\text{коврике}} \cdot \mathbf{u}_{\text{коврике}} = 0.2966 \cdot [0.35, 0.40] = [0.10381, 0.11864]$

Суммируем:
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{h}} =
[-0.039655 + 0.03501 + 0.065775 + 0.10381,\
-0.07931 + 0.04668 + 0.07893 + 0.11864]
$$
$$
= [0.16494, 0.16494]
$$



### 5.4. Обновление входного эмбеддинга $\mathbf{v}_{w_t}$

$$
\mathbf{v}_{\text{сидит}}^{\text{new}} = \mathbf{v}_{\text{сидит}}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{h}}
$$
$$
= [0.5, 0.7] - 0.01 \cdot [0.16494, 0.16494] = [0.5, 0.7] - [0.0016494, 0.0016494]
$$
$$
\mathbf{v}_{\text{сидит}}^{\text{new}} \approx [0.49835, 0.69835]
$$



## 6. Итоги одной итерации

После одной итерации обучения на паре $(\text{«сидит»}, \text{«кот»})$:
- Эмбеддинг $\mathbf{v}_{\text{сидит}}$ немного уменьшился.
- Эмбеддинг $\mathbf{u}_{\text{кот}}$ изменился так, чтобы **увеличить** скалярное произведение с $\mathbf{v}_{\text{сидит}}$.
- Эмбеддинги других слов изменились, чтобы **уменьшить** их вероятность в контексте слова «сидит».

Этот процесс повторяется для всех пар в корпусе на протяжении многих эпох. Со временем модель формирует осмысленные векторные представления, где семантически близкие слова оказываются близки в векторном пространстве.


## 7. Краткое упоминание Negative Sampling

Если бы мы использовали **Negative Sampling** с $k=1$ и отрицательным примером $w_{\text{neg}} = \text{«на»}$, то:

- Функция потерь:
  $$
  \mathcal{L}_{\text{NS}} = -\log \sigma(\mathbf{v}_{\text{сидит}}^\top \mathbf{u}_{\text{кот}}) - \log \sigma(-\mathbf{v}_{\text{сидит}}^\top \mathbf{u}_{\text{на}})
  $$

- Обновлялись бы **только**:
  - $\mathbf{v}_{\text{сидит}}$,
  - $\mathbf{u}_{\text{кот}}$,
  - $\mathbf{u}_{\text{на}}$.

- Не требовалось бы вычислять Softmax по всему словарю и обновлять все 4 выходных эмбеддинга.

✅ **Преимущество**: обучение становится на порядки быстрее, особенно при большом $V$.



# Математическая архитектура Continuous Bag-of-Words (CBOW)

## Введение: Векторные представления слов в обработке естественного языка

В современной обработке естественного языка (NLP) способность компьютера «понимать» смысл слов и их взаимосвязи является фундаментальной. Традиционные методы представления слов, такие как one-hot кодирование, где каждое уникальное слово в словаре представлено вектором, состоящим из одной единицы и множества нулей, страдают от двух основных недостатков:

1. **Разреженность**: Векторы становятся чрезвычайно длинными для больших словарей, что приводит к неэффективному использованию памяти и вычислительных ресурсов.  
2. **Отсутствие семантики**: One-hot векторы не несут никакой информации о смысловых или синтаксических отношениях между словами. Например, векторы для слов «король» и «королева» будут ортогональны, что не отражает их очевидной смысловой близости.

Для преодоления этих ограничений были разработаны векторные представления слов (word embeddings) или векторные вложения слов. Это плотные, низкоразмерные векторы действительных чисел, которые кодируют семантические и синтаксические свойства слов. Основная идея заключается в том, что слова, появляющиеся в схожих контекстах, имеют схожие значения и, следовательно, должны иметь схожие векторные представления в многомерном пространстве. Например, в хорошо обученном пространстве вложений вектор для слова «король» будет находиться близко к вектору «королева», и векторная операция «король» − «мужчина» + «женщина» будет аппроксимировать вектор «королева».

Одной из наиболее влиятельных моделей для обучения таких векторных представлений является Continuous Bag-of-Words (CBOW), предложенная Томашем Миколовым и его коллегами из Google в 2013 году. CBOW относится к классу моделей Word2Vec и представляет собой эффективный алгоритм для получения высококачественных векторных вложений из больших текстовых корпусов.

## 1. Основная концепция CBOW

Центральная идея CBOW заключается в предсказании целевого слова на основе его окружающего контекста. Модель принимает набор слов, находящихся в определённом окне вокруг интересующего слова (целевого слова), и использует их для прогнозирования самого целевого слова.

Рассмотрим пример предложения: «Собака гонится за кошкой по двору».  
Если «кошкой» является нашим целевым словом, то контекстными словами могут быть «Собака», «гонится», «за», «по», «двору» (в зависимости от размера окна контекста). Модель CBOW принимает эти контекстные слова в качестве входных данных и пытается предсказать «кошкой» как выход.

Процесс обучения CBOW можно резюмировать следующим образом:  
1. Формирование пар (контекст, целевое слово): из обучающего текстового корпуса извлекаются пары, где контекст состоит из слов, окружающих целевое слово в пределах заданного окна.  
2. Прямой проход (Forward Pass): модель обрабатывает входные контекстные слова, агрегирует их представления и вычисляет вероятности для каждого слова в словаре быть целевым словом.  
3. Вычисление потерь: сравнивается предсказанное распределение вероятностей с истинным целевым словом, и вычисляется ошибка предсказания.  
4. Обратный проход (Backpropagation): ошибка распространяется обратно через сеть, и веса модели (которые и являются векторными представлениями слов) корректируются таким образом, чтобы уменьшить ошибку в будущих предсказаниях.

Повторяя этот процесс многократно на обширном текстовом корпусе, модель учится эффективно кодировать семантические и синтаксические отношения в векторных представлениях слов. Чем чаще слово встречается в определённом контексте, тем сильнее модель «запоминает» эту связь, что приводит к более точным и значимым векторным вложениям.

## 2. Архитектура нейронной сети CBOW

Архитектура CBOW представляет собой относительно простую нейронную сеть, состоящую из трёх основных слоёв:

1. **Входной слой (Input Layer)**: этот слой принимает one-hot векторы контекстных слов. Если размер окна контекста составляет $C$ слов до и $C$ слов после целевого слова, то на вход подаётся $2C$ one-hot векторов.  
2. **Проекционный (скрытый) слой (Projection/Hidden Layer)**: этот слой является ключевым для создания векторных вложений. Он усредняет one-hot векторы контекстных слов и проецирует их в низкоразмерное векторное пространство. Размерность этого пространства, обозначаемая $N$, определяет размерность получаемых векторных вложений слов. В отличие от традиционных скрытых слоёв в нейронных сетях, здесь отсутствует нелинейная функция активации.  
3. **Выходной слой (Output Layer)**: этот слой имеет размер, равный размеру словаря ($V$). Он генерирует вектор оценок (логитов) для каждого слова в словаре, которые затем преобразуются в вероятности с помощью функции Softmax, указывающие на вероятность того, что каждое слово является целевым, учитывая входной контекст.




# 3. Детальная математическая формулировка

Для полного понимания работы CBOW необходимо подробно рассмотреть математические операции, происходящие на каждом этапе.  
Пусть $V$ — размер нашего словаря (общее количество уникальных слов), а $N$ — желаемая размерность векторных вложений (например, 100, 300).

## 3.1. Входной слой: One-Hot кодирование контекстных слов

Каждое слово в словаре представлено уникальным one-hot вектором. One-hot вектор для $i$-го слова $w_i$ — это вектор-столбец размерности $V \times 1$, в котором $i$-й элемент равен 1, а все остальные элементы равны 0.

Например, если словарь содержит 4 слова: {"кот": 0, "сидит": 1, "на": 2, "коврике": 3}, то:  
- One-hot вектор для "кот" будет $\mathbf{x}_{\text{кот}} = [1, 0, 0, 0]^\top$.  
- One-hot вектор для "сидит" будет $\mathbf{x}_{\text{сидит}} = [0, 1, 0, 0]^\top$.

Для данного обучающего примера, состоящего из целевого слова $w_c$ и его контекстных слов, мы имеем $2C$ one-hot векторов контекста: $\mathbf{x}_{c-C+1}, \dots, \mathbf{x}_{c-1}, \mathbf{x}_{c+1}, \dots, \mathbf{x}_{c+C}$.

## 3.2. Прямой проход (Forward Pass): От входного к проекционному слою

На этом этапе происходит преобразование разреженных one-hot векторов в плотные векторные представления и их агрегация.

### Матрица входных весов $W$

Между входным и проекционным слоем находится матрица весов $W \in \mathbb{R}^{V \times N}$. Каждая строка этой матрицы $W_i \in \mathbb{R}^{1 \times N}$ представляет собой $N$-мерное векторное вложение $i$-го слова в словаре, когда оно выступает в роли контекстного слова. Эту матрицу можно рассматривать как «таблицу поиска», где по индексу слова мы получаем его векторное представление.

### Получение векторного представления контекстного слова ($\mathbf{v}_j$)

Для каждого из $2C$ контекстных слов, представленных one-hot вектором $\mathbf{x}_j$, его плотное векторное представление $\mathbf{v}_j$ вычисляется путём умножения транспонированного one-hot вектора на матрицу $W$:
$$
\mathbf{v}_j = \mathbf{x}_j^\top W
$$
- $\mathbf{x}_j^\top$: транспонированный one-hot вектор $j$-го контекстного слова, имеющий размерность $1 \times V$.  
- $W$: матрица входных весов, размерность $V \times N$.  
- $\mathbf{v}_j$: векторное представление $j$-го контекстного слова, размерность $1 \times N$.

**Подробное объяснение**: Поскольку $\mathbf{x}_j$ является one-hot вектором, умножение $\mathbf{x}_j^\top W$ является эффективным способом извлечения соответствующей строки из матрицы $W$. Если $\mathbf{x}_j$ соответствует $k$-му слову в словаре, то $\mathbf{v}_j$ будет $k$-й строкой матрицы $W$.

### Усреднение контекстных векторов ($\mathbf{h}$)

После получения векторных представлений для всех $2C$ контекстных слов ($\mathbf{v}_1, \mathbf{v}_2, \dots, \mathbf{v}_{2C}$), CBOW усредняет эти векторы для формирования единого контекстного вектора $\mathbf{h}$:
$$
\mathbf{h} = \frac{1}{2C} \sum_{j=1}^{2C} \mathbf{v}_j = \frac{1}{2C} \sum_{j=1}^{2C} \mathbf{x}_j^\top W
$$
- $2C$: общее количество контекстных слов в текущем окне.  
- $\sum_{j=1}^{2C} \mathbf{v}_j$: сумма всех $2C$ векторных представлений контекстных слов.  
- $\mathbf{h}$: усреднённый контекстный вектор, размерность $1 \times N$.

**Подробное объяснение**: Вектор $\mathbf{h}$ представляет собой агрегированное, плотное представление всего контекста, окружающего целевое слово. Усреднение позволяет модели учитывать вклад каждого контекстного слова в равной степени, формируя единое, семантически насыщенное представление окружения. Этот вектор $\mathbf{h}$ является выходом проекционного слоя. Важно отметить, что в оригинальной архитектуре CBOW этот «скрытый» слой не имеет нелинейной функции активации; он выполняет только линейную проекцию и усреднение.

## 3.3. Прямой проход (Forward Pass): От проекционного к выходному слою

Теперь, имея контекстный вектор $\mathbf{h}$, модель должна предсказать целевое слово.

### Матрица выходных весов $W'$

Между проекционным и выходным слоями находится вторая матрица весов $W' \in \mathbb{R}^{N \times V}$. Каждая колонка этой матрицы $W'_k \in \mathbb{R}^{N \times 1}$ представляет собой $N$-мерное векторное вложение $k$-го слова в словаре, когда оно выступает в роли целевого слова.

### Вычисление оценок $u$ (логитов)

Контекстный вектор $\mathbf{h}$ умножается на транспонированную матрицу $W'^\top$ для получения вектора оценок $u$ для каждого слова в словаре:
$$
\mathbf{u} = W'^\top \mathbf{h}
$$
- $W'^\top$: транспонированная матрица $W'$, размерность $V \times N$.  
- $\mathbf{h}$: контекстный вектор, размерность $N \times 1$.  
- $\mathbf{u}$: вектор оценок (логитов), размерность $V \times 1$.

**Подробное объяснение**: Каждый элемент $u_k$ в векторе $\mathbf{u}$ является «оценкой» (или «логитом») того, насколько вероятно $k$-е слово в словаре является истинным целевым словом для данного контекста. Эта оценка вычисляется как скалярное произведение $k$-й строки $W'^\top$ (которая является $k$-й колонкой $W'$) и вектора $\mathbf{h}$:
$$
u_k = (W'_k)^\top \mathbf{h}
$$
где $(W'_k)^\top$ — это транспонированный вектор $k$-й колонки матрицы $W'$. Высокое значение $u_k$ указывает на то, что модель считает $k$-е слово весьма вероятным целевым словом в заданном контексте.

## 3.4. Прямой проход (Forward Pass): Выходной слой (Softmax)

Оценки $u_k$ могут принимать любые действительные значения. Для преобразования их в вероятности, которые суммируются к 1, используется функция Softmax.

### Функция Softmax

Вероятность $P(w_k \mid \text{контекст})$ того, что $k$-е слово является целевым словом, вычисляется следующим образом:
$$
P(w_k \mid \text{контекст}) = \frac{\exp(u_k)}{\sum_{j=1}^{V} \exp(u_j)}
$$
- $\exp(u_k)$: экспоненциальная функция от оценки $u_k$. Использование экспоненты гарантирует, что все значения будут положительными, а также усиливает различия между оценками, делая большие оценки значительно более доминирующими.  
- $\sum_{j=1}^{V} \exp(u_j)$: сумма экспонент всех оценок для каждого слова в словаре. Этот член является нормализующим множителем, который обеспечивает, чтобы сумма всех выходных вероятностей для всех слов в словаре была равна 1.

**Подробное объяснение**: Функция Softmax преобразует произвольные действительные числа (логиты) в дискретное распределение вероятностей. Она «сглаживает» оценки, делая их интерпретируемыми как вероятности. Слово с наибольшей оценкой $u_k$ получит наибольшую вероятность, но все остальные слова также будут иметь ненулевые вероятности, отражая степень неопределённости предсказания модели.

## 3.5. Функция потерь: Кросс-энтропия

Для обучения модели необходимо измерить, насколько «плохи» текущие предсказания. Для задач классификации, таких как предсказание целевого слова, широко используется функция потерь кросс-энтропии.

Пусть $w_o$ — истинное целевое слово (слово с индексом $o$ в словаре). Его one-hot представление $y_o$ будет иметь 1 на $o$-й позиции и 0 на всех остальных.

Функция потерь $E$ для одного обучающего примера определяется как отрицательный логарифм вероятности истинного целевого слова:
$$
E = -\log P(w_o \mid \text{контекст}) = -\log \left( \frac{\exp(u_o)}{\sum_{j=1}^{V} \exp(u_j)} \right)
$$
- $P(w_o \mid \text{контекст})$: предсказанная вероятность истинного целевого слова $w_o$ для данного контекста.  
- $\log$: натуральный логарифм.  
- $-$: отрицательный знак, поскольку мы стремимся минимизировать потери, а логарифм вероятности (которая находится в диапазоне от 0 до 1) будет отрицательным или нулевым.

**Подробное объяснение**: Функция потерь кросс-энтропии наказывает модель тем сильнее, чем ниже предсказанная вероятность истинного целевого слова. Если модель предсказывает правильное слово с вероятностью, близкой к 1, потери будут минимальны (близки к 0). Если же вероятность истинного слова близка к 0, потери будут очень большими (стремящимися к бесконечности). Цель обучения CBOW заключается в минимизации этой функции потерь, что эквивалентно максимизации логарифмической правдоподобности правильного предсказания.

# 3.6. Обратный проход (Backpropagation) и оптимизация: Градиентный спуск

Для минимизации функции потерь $E$ мы используем алгоритм градиентного спуска. Этот итеративный алгоритм обновляет веса модели ($W$ и $W'$) в направлении, противоположном градиенту функции потерь. Градиент указывает направление наибольшего увеличения функции, поэтому движение в противоположном направлении ведёт к её уменьшению.

### Общее правило обновления весов:
$$
\text{новые веса} = \text{старые веса} - \eta \cdot \nabla E
$$
- $\eta$ (скорость обучения, *learning rate*): небольшой положительный скаляр, который определяет размер шага при каждом обновлении весов. Выбор адекватной скорости обучения критичен: слишком большая $\eta$ может привести к «перепрыгиванию» через оптимальный минимум, а слишком маленькая — к чрезмерно медленной сходимости.  
- $\nabla E$: градиент функции потерь $E$ по отношению к конкретным весам. Это вектор, указывающий направление наиболее крутого подъёма функции $E$.

Вычисление градиентов для матриц $W$ и $W'$ является центральной частью обратного распространения ошибки и требует применения правила цепи (chain rule) из дифференциального исчисления.

## Градиенты для $W'$ (веса выходного слоя)

Начнём с вычисления градиента функции потерь по отношению к оценкам $u_k$:
$$
\frac{\partial E}{\partial u_k} = P(w_k \mid \text{контекст}) - y_k
$$
- $P(w_k \mid \text{контекст})$: предсказанная вероятность $k$-го слова.  
- $y_k$: истинное значение (1, если $k$-е слово является целевым; 0 в противном случае).

**Подробное объяснение**: Этот элегантный результат является прямым следствием использования функции Softmax с кросс-энтропийной потерей. Разница $(P(w_k \mid \text{контекст}) - y_k)$ представляет собой «ошибку предсказания» для $k$-го слова. Если предсказанная вероятность $P(w_k \mid \text{контекст})$ для истинного слова (где $y_k = 1$) высока, ошибка будет близка к 0. Если она низка, ошибка будет отрицательной и большой по модулю, что указывает на необходимость увеличения весов, ведущих к этому слову. Для неправильных слов (где $y_k = 0$), если $P(w_k \mid \text{контекст})$ высока, ошибка будет положительной и большой, указывая на необходимость уменьшения весов, ведущих к этому слову.

Теперь, используя правило цепи, мы можем найти градиент для $\mathbf{w}'_k$ (векторного представления $k$-го слова в $W'$, то есть $k$-й колонки $W'$):
$$
\frac{\partial E}{\partial \mathbf{w}'_k} = \frac{\partial E}{\partial u_k} \cdot \frac{\partial u_k}{\partial \mathbf{w}'_k} = (P(w_k \mid \text{контекст}) - y_k) \cdot \mathbf{h}
$$
- $\frac{\partial u_k}{\partial \mathbf{w}'_k}$: градиент $u_k$ по $\mathbf{w}'_k$. Поскольку $u_k = (\mathbf{w}'_k)^\top \mathbf{h}$, то $\frac{\partial u_k}{\partial \mathbf{w}'_k} = \mathbf{h}$.

### Обновление весов $W'$

Обновление весов для каждого векторного представления целевого слова $\mathbf{w}'_k$ происходит по формуле:
$$
\mathbf{w}'_k^{\text{новое}} = \mathbf{w}'_k^{\text{старое}} - \eta \cdot (P(w_k \mid \text{контекст}) - y_k) \cdot \mathbf{h}
$$

**Подробное объяснение**: Это обновление применяется для всех $V$ слов в словаре. Для каждого слова $w_k$ его векторное представление $\mathbf{w}'_k$ корректируется пропорционально ошибке предсказания для этого слова и контекстному вектору $\mathbf{h}$. Если ошибка положительна (модель переоценила вероятность слова), $\mathbf{w}'_k$ будет сдвигаться в направлении, противоположном $\mathbf{h}$. Если ошибка отрицательна (модель недооценила вероятность слова), $\mathbf{w}'_k$ будет сдвигаться в том же направлении, что и $\mathbf{h}$.

## Градиенты для $W$ (веса входного слоя)

Распространение ошибки на входную матрицу $W$ требует нескольких шагов, так как $W$ влияет на $\mathbf{h}$, который затем влияет на $\mathbf{u}$, который, в свою очередь, влияет на $E$.

Сначала вычисляем градиент по $\mathbf{h}$ (вектору контекста), который представляет собой сумму взвешенных ошибок от выходного слоя:
$$
\frac{\partial E}{\partial \mathbf{h}} = \sum_{k=1}^{V} \frac{\partial E}{\partial u_k} \cdot \frac{\partial u_k}{\partial \mathbf{h}} = \sum_{k=1}^{V} (P(w_k \mid \text{контекст}) - y_k) \cdot \mathbf{w}'_k
$$
- $\frac{\partial u_k}{\partial \mathbf{h}}$: градиент $u_k$ по $\mathbf{h}$. Поскольку $u_k = (\mathbf{w}'_k)^\top \mathbf{h}$, то $\frac{\partial u_k}{\partial \mathbf{h}} = \mathbf{w}'_k$.  
- $\sum_{k=1}^{V}$: суммирование по всем словам в словаре, так как $\mathbf{h}$ влияет на оценки всех слов.

Пусть $\mathbf{E}_H = \frac{\partial E}{\partial \mathbf{h}}$. Этот вектор $\mathbf{E}_H$ представляет собой агрегированную ошибку, которая распространяется обратно в проекционный слой. Он показывает, как изменение контекстного вектора $\mathbf{h}$ повлияет на общую функцию потерь.

Теперь, для каждого контекстного слова $\mathbf{x}_j$, его векторное представление $\mathbf{v}_j$ участвует в вычислении $\mathbf{h}$. Напомним, что:
$$
\mathbf{h} = \frac{1}{2C} \sum_{j=1}^{2C} \mathbf{v}_j
$$

### Градиент для $\mathbf{v}_j$ (векторного представления $j$-го контекстного слова):
$$
\frac{\partial E}{\partial \mathbf{v}_j} = \frac{\partial E}{\partial \mathbf{h}} \cdot \frac{\partial \mathbf{h}}{\partial \mathbf{v}_j} = \mathbf{E}_H \cdot \frac{1}{2C}
$$
- $\frac{\partial \mathbf{h}}{\partial \mathbf{v}_j}$: градиент $\mathbf{h}$ по $\mathbf{v}_j$. Поскольку $\mathbf{h}$ является средним значением всех $\mathbf{v}_j$, производная $\mathbf{h}$ по любому конкретному $\mathbf{v}_j$ будет $\frac{1}{2C}$.

**Подробное объяснение**: Этот шаг равномерно распределяет агрегированную ошибку $\mathbf{E}_H$ между всеми контекстными векторами $\mathbf{v}_j$, которые внесли вклад в формирование $\mathbf{h}$.

Наконец, градиент для $W$ (в частности, для строки $W_i$, соответствующей $i$-му слову в словаре, если это слово $w_i$ было одним из контекстных слов $\mathbf{x}_j$):
$$
\frac{\partial E}{\partial W_i} = \sum_{j : \mathbf{x}_j \text{ is } w_i} \frac{\partial E}{\partial \mathbf{v}_j} \cdot \frac{\partial \mathbf{v}_j}{\partial W_i} = \sum_{j : \mathbf{x}_j \text{ is } w_i} \frac{1}{2C} \mathbf{E}_H \cdot \mathbf{x}_j
$$
- $\frac{\partial \mathbf{v}_j}{\partial W_i}$: градиент $\mathbf{v}_j$ по $W_i$. Поскольку $\mathbf{v}_j = \mathbf{x}_j^\top W$, и $\mathbf{x}_j$ является one-hot вектором, то $\frac{\partial \mathbf{v}_j}{\partial W_i}$ будет ненулевым только для той строки $W_i$, которая соответствует слову $\mathbf{x}_j$. В этом случае производная эквивалентна $\mathbf{x}_j$ (вектор-столбец), что при умножении на скаляр и даёт вклад в обновление строки $W_i$.

### Обновление весов $W$

Обновление весов для каждой строки $W_i$ (т.е. векторного представления контекстного слова) происходит по формуле:
$$
W_i^{\text{новое}} = W_i^{\text{старое}} - \eta \cdot \frac{1}{2C} \mathbf{E}_H
$$

**Подробное объяснение**: Это обновление применяется только к тем строкам матрицы $W$, которые соответствуют словам, присутствующим в текущем контексте. Каждая такая строка $W_i$ корректируется пропорционально обратно распространённой ошибке $\mathbf{E}_H$, делённой на количество контекстных слов $2C$.



# 4. Заключение

Архитектура Continuous Bag-of-Words (CBOW), несмотря на свою структурную простоту, является чрезвычайно эффективным и мощным инструментом для создания семантически насыщенных векторных представлений слов. Глубокое понимание математических принципов, лежащих в её основе — от начального one-hot кодирования и усреднения контекстных векторов до применения функции Softmax для получения вероятностей и итеративного обновления весов посредством градиентного спуска — позволяет получить всестороннее представление о том, как нейронные сети способны «понимать» и кодировать значения слов из их лингвистического окружения.

Ключевые концепции, которые следует прочно усвоить:
- **Принцип предсказания**: CBOW учится предсказывать целевое слово на основе его контекстных слов.  
- **Две матрицы весов ($W$ и $W'$)**: Эти матрицы содержат обучаемые векторные представления слов. $W$ используется, когда слова выступают в роли контекста, а $W'$ — когда они являются целевыми словами. В идеале, после завершения обучения, эти матрицы будут содержать очень схожие, высококачественные векторные вложения. Для практического использования часто выбирают одну из них или усредняют обе.  
- **Усреднение контекста**: Проекционный слой CBOW выполняет простую, но эффективную операцию усреднения векторных представлений всех контекстных слов, формируя единый контекстный вектор.  
- **Функция Softmax**: необходима для преобразования произвольных числовых оценок в корректное распределение вероятностей, что позволяет интерпретировать выход модели как вероятность принадлежности к каждому слову в словаре.  
- **Градиентный спуск**: фундаментальный алгоритм оптимизации, используемый для итеративной настройки весов модели с целью минимизации функции потерь и, как следствие, повышения точности предсказаний.

Полученные с помощью CBOW векторные вложения слов являются ценным ресурсом и находят широкое применение в различных задачах NLP, включая машинный перевод, анализ настроений, вопросно-ответные системы, классификацию текста и многие другие, значительно повышая их производительность и способность к семантическому анализу.




# Аналитический пример математической архитектуры Continuous Bag-of-Words (CBOW)

Этот раздел посвящён детальному аналитическому примеру работы архитектуры CBOW, шаг за шагом демонстрируя все вычисления прямого и обратного проходов. Цель состоит в том, чтобы сделать каждую формулу и операцию максимально понятной, позволяя любому читателю полностью освоить математические основы CBOW.

## 1. Исходные данные и параметры

Для нашего примера мы определим минимальный набор данных и параметров:

- **Словарь ($V$)**: {"я": 0, "люблю": 1, "кошек": 2, "собак": 3}. Размер словаря $V = 4$.  
- **Размерность векторных вложений ($N$)**: $N = 2$. Это означает, что каждое слово будет представлено 2-мерным вектором.  
- **Размер окна контекста ($C$)**: $C = 1$. Это означает, что мы берём одно слово до и одно слово после целевого слова. Общее количество контекстных слов $2C = 2$.  
- **Обучающий пример**: Предложение "я люблю кошек собак".  
  - Целевое слово ($w_o$): "кошек" (индекс 2). Его one-hot вектор $\mathbf{y}_o = [0, 0, 1, 0]^\top$.  
  - Контекстные слова: "люблю" (индекс 1) и "собак" (индекс 3).  
- **Скорость обучения ($\eta$)**: $\eta = 0.01$.

### Инициализация матриц весов

Мы инициализируем матрицы весов $W$ и $W'$ случайными значениями. Для простоты вычислений выберем небольшие, легко отслеживаемые числа.

#### 1. Матрица входных весов $W \in \mathbb{R}^{V \times N}$

Каждая строка $W_i$ соответствует векторному представлению $i$-го слова, когда оно является контекстным.
$$
W =
\begin{bmatrix}
W_0 \\
W_1 \\
W_2 \\
W_3
\end{bmatrix}
=
\begin{bmatrix}
0.1 & 0.3 \\
0.5 & 0.7 \\
0.2 & 0.4 \\
0.6 & 0.8
\end{bmatrix}
$$
- $W_0$: вектор для "я"  
- $W_1$: вектор для "люблю"  
- $W_2$: вектор для "кошек"  
- $W_3$: вектор для "собак"


#### 2. Матрица выходных весов $W' \in \mathbb{R}^{N \times V}$

Каждый столбец $W'_k$ соответствует векторному представлению $k$-го слова, когда оно является целевым.
$$
W' =
\begin{bmatrix}
0.9 & 0.8 & 0.7 & 0.6 \\
0.5 & 0.4 & 0.3 & 0.2
\end{bmatrix}
\quad
\text{или} \quad
W' = (W'_0\ W'_1\ W'_2\ W'_3)
$$
- $W'_0$: вектор для "я"  
- $W'_1$: вектор для "люблю"  
- $W'_2$: вектор для "кошек"  
- $W'_3$: вектор для "собак"



## 2. Прямой проход (Forward Pass)

Прямой проход включает вычисление предсказаний модели на основе текущих весов.

### 2.1. Входной слой: One-Hot кодирование контекстных слов

Контекстные слова: "люблю" (индекс 1) и "собак" (индекс 3).  
Их one-hot векторы:
- $\mathbf{x}_{\text{люблю}} = [0, 1, 0, 0]^\top$  
- $\mathbf{x}_{\text{собак}} = [0, 0, 0, 1]^\top$



### 2.2. От входного к проекционному слою

#### Получение векторного представления контекстного слова ($\mathbf{v}_j$)

Для каждого контекстного слова $\mathbf{x}_j$, его плотное векторное представление $\mathbf{v}_j$ извлекается из матрицы $W$ как $\mathbf{v}_j = \mathbf{x}_j^\top W$:

- Для "люблю" (индекс 1):  
  $$
  \mathbf{v}_{\text{люблю}} = \mathbf{x}_{\text{люблю}}^\top W = [0, 1, 0, 0]
  \begin{bmatrix}
  0.1 & 0.3 \\
  0.5 & 0.7 \\
  0.2 & 0.4 \\
  0.6 & 0.8
  \end{bmatrix}
  = [0.5, 0.7]
  $$

- Для "собак" (индекс 3):  
  $$
  \mathbf{v}_{\text{собак}} = \mathbf{x}_{\text{собак}}^\top W = [0, 0, 0, 1]
  \begin{bmatrix}
  0.1 & 0.3 \\
  0.5 & 0.7 \\
  0.2 & 0.4 \\
  0.6 & 0.8
  \end{bmatrix}
  = [0.6, 0.8]
  $$

#### Усреднение контекстных векторов ($\mathbf{h}$)

$$
\mathbf{h} = \frac{1}{2C} \sum_{j=1}^{2C} \mathbf{v}_j = \frac{1}{2} (\mathbf{v}_{\text{люблю}} + \mathbf{v}_{\text{собак}})
$$
$$
\mathbf{h} = \frac{1}{2} \left( [0.5, 0.7] + [0.6, 0.8] \right) = \frac{1}{2} [1.1, 1.5] = [0.55, 0.75]
$$

Таким образом, контекстный вектор $\mathbf{h} = [0.55, 0.75]^\top$.



### 2.3. От проекционного к выходному слою

#### Вычисление оценок $u$ (логитов)

Умножаем контекстный вектор $\mathbf{h}$ на транспонированную матрицу $W'^\top$:

$$
\mathbf{u} = W'^\top \mathbf{h}, \quad
W'^\top =
\begin{bmatrix}
0.9 & 0.5 \\
0.8 & 0.4 \\
0.7 & 0.3 \\
0.6 & 0.2
\end{bmatrix},
\quad
\mathbf{h} =
\begin{bmatrix}
0.55 \\
0.75
\end{bmatrix}
$$

Вычисляем каждый элемент $u_k = (W'_k)^\top \mathbf{h}$:

- $u_0$ (для "я"): $0.9 \cdot 0.55 + 0.5 \cdot 0.75 = 0.495 + 0.375 = 0.870$  
- $u_1$ (для "люблю"): $0.8 \cdot 0.55 + 0.4 \cdot 0.75 = 0.440 + 0.300 = 0.740$  
- $u_2$ (для "кошек"): $0.7 \cdot 0.55 + 0.3 \cdot 0.75 = 0.385 + 0.225 = 0.610$  
- $u_3$ (для "собак"): $0.6 \cdot 0.55 + 0.2 \cdot 0.75 = 0.330 + 0.150 = 0.480$

Таким образом, вектор оценок:
$$
\mathbf{u} = [0.870, 0.740, 0.610, 0.480]^\top
$$



### 2.4. Выходной слой (Softmax)

Преобразуем оценки $u_k$ в вероятности $P(w_k \mid \text{контекст})$ с помощью функции Softmax:
$$
P(w_k \mid \text{контекст}) = \frac{\exp(u_k)}{\sum_{j=0}^{3} \exp(u_j)}
$$

Вычислим экспоненты:
- $\exp(0.870) \approx 2.386$  
- $\exp(0.740) \approx 2.096$  
- $\exp(0.610) \approx 1.840$  
- $\exp(0.480) \approx 1.616$

Сумма:  
$$
\sum_j \exp(u_j) = 2.386 + 2.096 + 1.840 + 1.616 = 7.938
$$

Теперь вероятности:
- $P(\text{«я»} \mid \text{контекст}) = 2.386 / 7.938 \approx 0.3006$  
- $P(\text{«люблю»} \mid \text{контекст}) = 2.096 / 7.938 \approx 0.2641$  
- $P(\text{«кошек»} \mid \text{контекст}) = 1.840 / 7.938 \approx 0.2318$  
- $P(\text{«собак»} \mid \text{контекст}) = 1.616 / 7.938 \approx 0.2036$

Вектор предсказанных вероятностей:  
$$
\mathbf{P} = [0.3006, 0.2641, 0.2318, 0.2036]^\top
$$


### 2.5. Функция потерь: Кросс-энтропия

Истинное целевое слово $w_o$ — "кошек" (индекс 2). Его one-hot вектор $\mathbf{y}_o = [0, 0, 1, 0]^\top$.  
Функция потерь:
$$
E = -\log P(w_o \mid \text{контекст}) = -\log(0.2318) \approx -(-1.462) = 1.462
$$

Текущее значение функции потерь $E \approx 1.462$. Наша цель — уменьшить это значение.



## 3. Обратный проход (Backpropagation)

Обратный проход включает вычисление градиентов функции потерь по отношению к весам и их последующее обновление.

### 3.1. Градиенты для $W'$ (веса выходного слоя)

Сначала вычислим ошибку предсказания для каждого слова:
$$
\delta_k = \frac{\partial E}{\partial u_k} = P(w_k \mid \text{контекст}) - y_k
$$

- $\delta_0$ (для "я"): $0.3006 - 0 = 0.3006$  
- $\delta_1$ (для "люблю"): $0.2641 - 0 = 0.2641$  
- $\delta_2$ (для "кошек"): $0.2318 - 1 = -0.7682$  
- $\delta_3$ (для "собак"): $0.2036 - 0 = 0.2036$

Вектор ошибок: $\boldsymbol{\delta} = [0.3006, 0.2641, -0.7682, 0.2036]^\top$

Теперь вычислим градиент для каждого столбца $W'_k$:
$$
\frac{\partial E}{\partial W'_k} = \delta_k \cdot \mathbf{h}
$$

Напомним: $\mathbf{h} = [0.55, 0.75]^\top$

- $\frac{\partial E}{\partial W'_0} = 0.3006 \cdot [0.55, 0.75] = [0.16533, 0.22545]$  
- $\frac{\partial E}{\partial W'_1} = 0.2641 \cdot [0.55, 0.75] = [0.14526, 0.19808]$  
- $\frac{\partial E}{\partial W'_2} = -0.7682 \cdot [0.55, 0.75] = [-0.42251, -0.57615]$  
- $\frac{\partial E}{\partial W'_3} = 0.2036 \cdot [0.55, 0.75] = [0.11198, 0.15270]$

#### Обновление весов $W'$

$$
W'_k^{\text{новое}} = W'_k^{\text{старое}} - \eta \cdot \frac{\partial E}{\partial W'_k}
$$

Обновим $W'_2$ (для "кошек"):
- $W'_2^{\text{старое}} = [0.7, 0.3]^\top$  
- $W'_2^{\text{новое}} = [0.7, 0.3] - 0.01 \cdot [-0.42251, -0.57615] = [0.7 + 0.0042251, 0.3 + 0.0057615] = [0.704225, 0.305762]^\top$

Аналогичные обновления применяются к $W'_0$, $W'_1$, $W'_3$.



### 3.2. Градиенты для $W$ (веса входного слоя)

#### Вычисление агрегированной ошибки $E_H = \frac{\partial E}{\partial \mathbf{h}}$

$$
\frac{\partial E}{\partial \mathbf{h}} = \sum_{k=0}^{3} \delta_k \cdot W'_k
$$

- $\delta_0 \cdot W'_0 = 0.3006 \cdot [0.9, 0.5] = [0.27054, 0.15030]$  
- $\delta_1 \cdot W'_1 = 0.2641 \cdot [0.8, 0.4] = [0.21128, 0.10564]$  
- $\delta_2 \cdot W'_2 = -0.7682 \cdot [0.7, 0.3] = [-0.53774, -0.23046]$  
- $\delta_3 \cdot W'_3 = 0.2036 \cdot [0.6, 0.2] = [0.12216, 0.04072]$

Суммируем:
- $E_{H_x} = 0.27054 + 0.21128 - 0.53774 + 0.12216 = 0.06624$  
- $E_{H_y} = 0.15030 + 0.10564 - 0.23046 + 0.04072 = 0.06620$

Таким образом, $E_H \approx [0.0662, 0.0662]^\top$

#### Градиент для контекстных векторов $\mathbf{v}_j$

$$
\frac{\partial E}{\partial \mathbf{v}_j} = \frac{\partial E}{\partial \mathbf{h}} \cdot \frac{\partial \mathbf{h}}{\partial \mathbf{v}_j} = E_H \cdot \frac{1}{2C} = E_H \cdot 0.5
$$
$$
\frac{\partial E}{\partial \mathbf{v}_j} = [0.0662, 0.0662] \cdot 0.5 = [0.0331, 0.0331]
$$

#### Обновление весов $W$

$$
W_i^{\text{новое}} = W_i^{\text{старое}} - \eta \cdot \frac{\partial E}{\partial \mathbf{v}_j}
$$

- Для $W_1$ ("люблю"):  
  $W_1^{\text{старое}} = [0.5, 0.7]^\top$  
  $W_1^{\text{новое}} = [0.5, 0.7] - 0.01 \cdot [0.0331, 0.0331] = [0.499669, 0.699669]^\top$

- Для $W_3$ ("собак"):  
  $W_3^{\text{старое}} = [0.6, 0.8]^\top$  
  $W_3^{\text{новое}} = [0.6, 0.8] - 0.01 \cdot [0.0331, 0.0331] = [0.599669, 0.799669]^\top$

Строки $W_0$ и $W_2$ не обновляются, так как "я" и "кошек" не были контекстными словами в этом примере.



## 4. Заключение

Этот аналитический пример демонстрирует полный цикл прямого и обратного проходов в архитектуре CBOW для одного обучающего примера. Мы пошагово вычислили:
- Векторные представления контекстных слов.  
- Усреднённый контекстный вектор.  
- Оценки (логиты) для всех слов в словаре.  
- Вероятности предсказания целевого слова с помощью Softmax.  
- Значение функции потерь кросс-энтропии.  
- Градиенты для обновления весов в матрицах $W'$ и $W$.  
- Примеры обновлённых весов.

Этот процесс повторяется миллионы раз на большом текстовом корпусе. С каждым шагом веса $W$ и $W'$ постепенно корректируются, чтобы модель всё точнее предсказывала целевые слова на основе их контекста. В результате этих итераций формируются качественные векторные представления слов, которые кодируют семантические и синтаксические отношения между ними.




# Оптимизации Word2Vec

Для повышения эффективности обучения Word2Vec, особенно при работе с очень большими словарями, используются две основные оптимизации:

- **Иерархический Softmax (Hierarchical Softmax)**:  
  Вместо вычисления вероятностей для всех $V$ слов в выходном слое с помощью стандартного Softmax, иерархический Softmax использует **двоичное дерево Хаффмана**, в котором листьями являются слова словаря. Вероятность слова вычисляется как произведение вероятностей переходов по пути от корня дерева к соответствующему листу. Это снижает вычислительную сложность обновления весов с $O(V)$ до $O(\log V)$, что особенно эффективно при больших $V$.

- **Отрицательное сэмплирование (Negative Sampling)**:  
  Это более популярная и вычислительно эффективная оптимизация. Вместо многоклассовой классификации (предсказание одного истинного слова среди $V$), задача преобразуется в **бинарную классификацию**.  
  Для каждой положительной пары (целевое слово, контекстное слово) модель обучается предсказывать метку 1. Одновременно выбирается $k$ случайных слов (отрицательные примеры), не входящих в контекст, и для пар (целевое слово, отрицательное слово) модель обучается предсказывать метку 0.  
  Таким образом, на каждом шаге обновляются веса только для истинного контекстного слова и $k$ отрицательных слов, а не для всего словаря. Это резко снижает вычислительную нагрузку и ускоряет обучение.



#  Преимущества и недостатки Word2Vec (общие)

## Преимущества

- **Эффективность**:  
  Word2Vec способен обучаться на очень больших текстовых корпусах, генерируя качественные эмбеддинги за разумное время.

- **Семантические и синтаксические отношения**:  
  Эмбеддинги Word2Vec улавливают не только семантическую близость слов (например, «король» и «королева» находятся близко в векторном пространстве), но и линейные аналогии:  
  $$
  \mathbf{v}_{\text{король}} - \mathbf{v}_{\text{мужчина}} + \mathbf{v}_{\text{женщина}} \approx \mathbf{v}_{\text{королева}}
  $$

- **Плотные представления**:  
  В отличие от разреженных векторов (например, one-hot или TF-IDF), плотные эмбеддинги компактны, эффективны с точки зрения памяти и хорошо работают в составе последующих моделей машинного обучения.

- **Гибкость**:  
  Модель может использоваться как для обучения с нуля, так и для тонкой настройки (fine-tuning) предварительно обученных эмбеддингов в рамках конкретной задачи.

## Недостатки

- **Статические эмбеддинги**:  
  Каждое слово имеет **один фиксированный вектор**, независимо от контекста его употребления. Это приводит к следующим проблемам:
  - **Проблема полисемии**: модель не может различать разные значения одного и того же слова. Например, слово «банк» в значениях *финансовое учреждение* и *берег реки* будет представлено одним и тем же вектором.
  - **Отсутствие контекстной чувствительности**: Word2Vec не учитывает, как слово взаимодействует с другими словами в предложении. Это ограничивает его способность понимать нюансы смысла, идиомы, отрицания и другие контекстно-зависимые явления.

- **Неспособность к обучению на новых данных без полного переобучения**:  
  Новые слова, отсутствующие в исходном словаре (OOV — *out-of-vocabulary*), не имеют векторных представлений. Чтобы добавить их, требуется либо расширение словаря, либо полное переобучение модели на обновлённом корпусе, что неэффективно.



Несмотря на эти ограничения, **Word2Vec стал краеугольным камнем в NLP** и проложил путь для развития более сложных моделей, таких как **ELMo**, **BERT** и других, которые используют **контекстно-зависимые эмбеддинги** и решают многие из указанных проблем.



# GloVe: Глобальные векторы для представления слов

## 1. Введение в векторные представления слов (Word Embeddings)

В области обработки естественного языка (NLP) слова традиционно представлялись как дискретные символы. Однако такой подход не позволяет улавливать семантические и синтаксические отношения между словами. Например, слова «король» и «королева» имеют схожие значения, но их дискретные представления не отражают этого сходства.

Векторные представления слов, или **word embeddings**, решают эту проблему, отображая слова в плотные векторы вещественных чисел в многомерном пространстве. В этом пространстве слова с похожим значением или контекстом располагаются близко друг к другу. Это позволяет моделям машинного обучения лучше понимать язык и выполнять такие задачи, как машинный перевод, анализ настроений, вопросно-ответные системы и многие другие.

Существует два основных подхода к созданию векторных представлений:

1. **Прогнозные модели (prediction-based models)**:  
   Пример — Word2Vec (Skip-gram и CBOW). Они обучаются предсказывать слово по его контексту или контекст по слову.

2. **Модели, основанные на частоте встречаемости (count-based models)**:  
   Пример — Latent Semantic Analysis (LSA). Они анализируют статистику встречаемости слов в большом корпусе текстов.

Модель **GloVe (Global Vectors for Word Representation)**, разработанная Стэнфордским университетом, представляет собой гибридный подход, который пытается объединить преимущества обоих методов. Она использует глобальную статистику со-встречаемости слов (как в count-based моделях) для обучения векторных представлений, но при этом оптимизирует функцию потерь, похожую на те, что используются в прогнозных моделях.

Основная идея GloVe заключается в том, что отношения между словами могут быть закодированы в разностях их векторных представлений. Например:
$$
\mathbf{v}_{\text{король}} - \mathbf{v}_{\text{мужчина}} \approx \mathbf{v}_{\text{королева}} - \mathbf{v}_{\text{женщина}}
$$



## 2. Математические основы модели GloVe

### 2.1. Матрица со-встречаемости (Co-occurrence Matrix)

В основе GloVe лежит **матрица со-встречаемости** $X$. Элемент $X_{ij}$ этой матрицы представляет собой количество раз, когда слово $j$ встречается в контексте слова $i$ в заданном окне. Размер окна определяет, насколько далеко друг от друга могут находиться слова, чтобы считаться «со-встречающимися».

#### Пример:
Предположим, у нас есть корпус:
- «I like deep learning.»
- «I like NLP.»
- «I love learning.»

Используем окно размером 1 (рассматриваем только непосредственных соседей).

Матрица $X$ будет выглядеть примерно так:

| Word \ Context | I   | like | deep | learning | NLP | love |
|----------------|-----|------|------|----------|-----|------|
| I              | 0   | 2    | 0    | 0        | 0   | 0    |
| like           | 1   | 0    | 1    | 0        | 0   | 0    |
| deep           | 0   | 1    | 0    | 1        | 0   | 0    |
| learning       | 0   | 0    | 1    | 0        | 1   | 1    |
| NLP            | 0   | 0    | 0    | 1        | 0   | 0    |
| love           | 0   | 0    | 0    | 1        | 0   | 0    |

> **Примечание**: Обычно матрица со-встречаемости симметрична ($X_{ij} = X_{ji}$), если контекстное окно не учитывает направление. В GloVe часто используется симметричное окно, и $X_{ij}$ суммирует со-встречаемость $i$ с $j$ и $j$ с $i$. Также часто применяется затухающая функция веса со-встречаемости в зависимости от расстояния. Например, если слова находятся на расстоянии $d$, их со-встречаемость может учитываться с весом $1/d$.



### 2.2. Функция потерь GloVe (Objective Function)

Основная идея GloVe заключается в том, чтобы найти такие векторные представления слов $\mathbf{v}_i$ и $\mathbf{v}_j$ (а также контекстные векторы $\mathbf{v}'_j$), чтобы их скалярное произведение $\mathbf{v}_i^\top \mathbf{v}_j$ хорошо аппроксимировало логарифм частоты со-встречаемости $\log(X_{ij})$.

Функция потерь GloVe определяется следующим образом:
$$
J = \sum_{i=1}^{V} \sum_{j=1}^{V} f(X_{ij}) \left( \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij}) \right)^2
$$

Где:
- $V$ — размер словаря.
- $\mathbf{v}_i$ — векторное представление слова $i$.
- $\mathbf{v}_j$ — векторное представление слова $j$ (контекстный вектор). В GloVe для каждого слова обучаются два вектора: основной вектор $\mathbf{v}_i$ и контекстный вектор $\mathbf{v}'_j$. В конце обучения обычно используется сумма этих векторов ($\mathbf{v}_i + \mathbf{v}'_i$) или только основной вектор $\mathbf{v}_i$. Для простоты обозначим контекстный вектор как $\mathbf{v}_j$.
- $b_i$ и $b_j$ — скалярные смещения (bias terms) для слов $i$ и $j$ соответственно. Они учитывают общую частоту встречаемости слов, которая не улавливается скалярным произведением.
- $\log(X_{ij})$ — логарифм частоты со-встречаемости слова $j$ в контексте слова $i$. На практике используется $\log(1 + X_{ij})$, чтобы избежать $\log(0)$ для пар, которые никогда не встречаются.
- $f(X_{ij})$ — весовая функция (weighting function). Она предназначена для:
  - Присвоения нулевого веса парам, которые никогда не встречаются ($X_{ij} = 0$), чтобы они не влияли на функцию потерь.
  - Присвоения меньшего веса очень частым парам (например, «the the»), которые могут нести меньше семантической информации.
  - Присвоения большего веса редким, но информативным парам.

Типичная весовая функция $f(x)$ имеет вид:
$$
f(x) =
\begin{cases}
(x / x_{\text{max}})^\alpha & \text{если } x < x_{\text{max}} \\
1 & \text{если } x \geq x_{\text{max}}
\end{cases}
$$

Где:
- $x$ — это $X_{ij}$,
- $x_{\text{max}}$ — пороговое значение (например, 100). Пары с частотой со-встречаемости выше $x_{\text{max}}$ получают полный вес 1,
- $\alpha$ — параметр (обычно 0.75).



### 2.3. Интуиция за функцией потерь

Почему эта функция потерь работает?

Цель GloVe — найти векторные представления, которые кодируют **отношения между словами**, проявляющиеся в **отношениях частот со-встречаемости**.

Рассмотрим отношение вероятностей того, что слово $j$ встретится в контексте слов $i$ и $k$:
$$
\frac{P(j \mid i)}{P(j \mid k)}
$$
GloVe предполагает, что это отношение может быть выражено через разность векторов:
$$
\mathbf{v}_i^\top \mathbf{v}_j - \mathbf{v}_k^\top \mathbf{v}_j \approx \log\left(\frac{X_{ij}}{X_{kj}}\right) = \log(X_{ij}) - \log(X_{kj})
$$
Это можно переписать как:
$$
\mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j \approx \log(X_{ij})
$$
Таким образом, минимизация квадратичной разницы между скалярным произведением векторов (плюс смещения) и логарифмом частоты со-встречаемости заставляет модель учиться таким образом, чтобы векторные отношения отражали статистические отношения в корпусе.


## 3. Оптимизация модели GloVe: Обучение

Обучение модели GloVe сводится к минимизации функции потерь $J$ по всем параметрам: векторам слов $\mathbf{v}_i$, контекстным векторам $\mathbf{v}_j$ и смещениям $b_i$, $b_j$. Это достигается с использованием алгоритма **стохастического градиентного спуска (Stochastic Gradient Descent, SGD)** или его вариантов (например, AdaGrad, Adam).

В SGD параметры обновляются итеративно, делая небольшие шаги в направлении, противоположном градиенту функции потерь.

### 3.1. Прямой проход (Forward Pass)

Прямой проход для GloVe относительно прост, поскольку он не включает сложной нейронной сети. Для каждой пары слов $(i, j)$ с ненулевой частотой со-встречаемости $X_{ij}$:

1. **Извлечение параметров**: Получаем текущие векторные представления $\mathbf{v}_i$ и $\mathbf{v}_j$, а также смещения $b_i$ и $b_j$.
2. **Вычисление предсказанного значения**: Вычисляем скалярное произведение $\mathbf{v}_i^\top \mathbf{v}_j$ и добавляем смещения:
$$
   \text{prediction} = \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j
$$
3. **Вычисление целевого значения**: Вычисляем логарифм частоты со-встречаемости:
$$
   \text{target} = \log(X_{ij})
$$
   (На практике используется $\log(1 + X_{ij})$.)
4. **Вычисление ошибки**: Разница между предсказанным и целевым значением:
$$
   \text{error} = \text{prediction} - \text{target} = \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij})
$$
5. **Вычисление взвешенной квадратичной ошибки для данной пары**:
$$
   L_{ij} = f(X_{ij}) \cdot (\text{error})^2 = f(X_{ij}) \left( \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij}) \right)^2
$$
   Этот $L_{ij}$ является вкладом одной пары $(i, j)$ в общую функцию потерь $J$.


# 3.2. Обратное распространение ошибки (Backpropagation) и обновление параметров

Обратное распространение ошибки в GloVe заключается в вычислении частных производных функции потерь $J$ по каждому из обучаемых параметров ($\mathbf{v}_i$, $\mathbf{v}_j$, $b_i$, $b_j$). Затем эти градиенты используются для обновления параметров.

Для простоты рассмотрим вклад одной пары $(i, j)$ в функцию потерь:
$$
L_{ij} = f(X_{ij}) \left( \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij}) \right)^2
$$

Обозначим:
$$
E_{ij} = \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij})
$$
Тогда:
$$
L_{ij} = f(X_{ij}) E_{ij}^2
$$

Мы хотим найти градиенты:
$$
\frac{\partial L_{ij}}{\partial \mathbf{v}_i},\
\frac{\partial L_{ij}}{\partial \mathbf{v}_j},\
\frac{\partial L_{ij}}{\partial b_i},\
\frac{\partial L_{ij}}{\partial b_j}
$$



### 3.2.1. Градиент по $\mathbf{v}_i$

Применяем правило цепи:
$$
\frac{\partial L_{ij}}{\partial \mathbf{v}_i} = \frac{\partial L_{ij}}{\partial E_{ij}} \cdot \frac{\partial E_{ij}}{\partial \mathbf{v}_i}
$$

Сначала найдём:
$$
\frac{\partial L_{ij}}{\partial E_{ij}} = \frac{\partial}{\partial E_{ij}} \left( f(X_{ij}) E_{ij}^2 \right) = 2 f(X_{ij}) E_{ij}
$$

Затем:
$$
\frac{\partial E_{ij}}{\partial \mathbf{v}_i} = \frac{\partial}{\partial \mathbf{v}_i} \left( \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij}) \right) = \mathbf{v}_j
$$

Таким образом, градиент по $\mathbf{v}_i$:
$$
\frac{\partial L_{ij}}{\partial \mathbf{v}_i} = 2 f(X_{ij}) E_{ij} \cdot \mathbf{v}_j
$$



### 3.2.2. Градиент по $\mathbf{v}_j$

Аналогично:
$$
\frac{\partial L_{ij}}{\partial \mathbf{v}_j} = \frac{\partial L_{ij}}{\partial E_{ij}} \cdot \frac{\partial E_{ij}}{\partial \mathbf{v}_j}
$$

Мы уже знаем:
$$
\frac{\partial L_{ij}}{\partial E_{ij}} = 2 f(X_{ij}) E_{ij}
$$

Теперь:
$$
\frac{\partial E_{ij}}{\partial \mathbf{v}_j} = \frac{\partial}{\partial \mathbf{v}_j} \left( \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij}) \right) = \mathbf{v}_i
$$

Таким образом, градиент по $\mathbf{v}_j$:
$$
\frac{\partial L_{ij}}{\partial \mathbf{v}_j} = 2 f(X_{ij}) E_{ij} \cdot \mathbf{v}_i
$$



### 3.2.3. Градиент по $b_i$

$$
\frac{\partial L_{ij}}{\partial b_i} = \frac{\partial L_{ij}}{\partial E_{ij}} \cdot \frac{\partial E_{ij}}{\partial b_i}
$$

$$
\frac{\partial E_{ij}}{\partial b_i} = \frac{\partial}{\partial b_i} \left( \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij}) \right) = 1
$$

Следовательно:
$$
\frac{\partial L_{ij}}{\partial b_i} = 2 f(X_{ij}) E_{ij}
$$



### 3.2.4. Градиент по $b_j$

Аналогично:
$$
\frac{\partial L_{ij}}{\partial b_j} = \frac{\partial L_{ij}}{\partial E_{ij}} \cdot \frac{\partial E_{ij}}{\partial b_j}
$$

$$
\frac{\partial E_{ij}}{\partial b_j} = 1
$$

Следовательно:
$$
\frac{\partial L_{ij}}{\partial b_j} = 2 f(X_{ij}) E_{ij}
$$



### 3.2.5. Обновление параметров

После вычисления градиентов для каждой пары $(i, j)$ с ненулевой частотой со-встречаемости, параметры обновляются по правилу градиентного спуска:
$$
\mathbf{v}_i^{\text{new}} = \mathbf{v}_i^{\text{old}} - \eta \cdot \frac{\partial L_{ij}}{\partial \mathbf{v}_i}
$$
$$
\mathbf{v}_j^{\text{new}} = \mathbf{v}_j^{\text{old}} - \eta \cdot \frac{\partial L_{ij}}{\partial \mathbf{v}_j}
$$
$$
b_i^{\text{new}} = b_i^{\text{old}} - \eta \cdot \frac{\partial L_{ij}}{\partial b_i}
$$
$$
b_j^{\text{new}} = b_j^{\text{old}} - \eta \cdot \frac{\partial L_{ij}}{\partial b_j}
$$

Где $\eta$ (скорость обучения, *learning rate*) — небольшой положительный скаляр, определяющий размер шага обновления.

На практике вместо обработки каждой пары $(i, j)$ по отдельности (что может быть очень медленно для больших корпусов) часто используется **мини-пакетный градиентный спуск (mini-batch gradient descent)**, где градиенты усредняются по небольшому набору пар перед обновлением параметров.



# 4. Алгоритм обучения GloVe (пошагово)

1. **Сбор корпуса текстов**:  
   Подготовьте большой текстовый корпус, на котором будет обучаться модель.

2. **Построение словаря**:  
   Извлеките все уникальные слова из корпуса и создайте словарь (vocabulary). Каждому слову присваивается уникальный индекс.

3. **Построение матрицы со-встречаемости $X$**:
   - Инициализируйте матрицу $X$ размером $V \times V$ (где $V$ — размер словаря) нулями.
   - Для каждого слова в корпусе пройдитесь по его контекстному окну (например, 5 слов влево и 5 слов вправо).
   - Для каждой пары со-встречающихся слов $(i, j)$ в окне увеличьте $X_{ij}$ на 1 (или на $1/d$, где $d$ — расстояние между словами).
   - Примените пороговое значение для редких слов, отбрасывая их или заменяя на токен `<UNK>`.

4. **Инициализация параметров**:
   - Инициализируйте векторные представления слов $\mathbf{v}_i$ и контекстные векторы $\mathbf{v}'_j$ (они могут быть разными или одинаковыми в начале) случайными малыми значениями. Размерность векторов (например, 50, 100, 300) является гиперпараметром.
   - Инициализируйте скалярные смещения $b_i$ и $b_j$ нулями или случайными малыми значениями.

5. **Итеративное обучение (эпохи)**:  
   Повторяйте следующие шаги в течение заданного количества эпох:
   - **Перемешивание данных**: Перемешайте список всех пар $(i, j)$ с ненулевой частотой со-встречаемости.
   - **Итерация по парам**: Для каждой пары $(i, j)$ из перемешанного списка:
     - **Прямой проход**:
       - Вычислите $E_{ij} = \mathbf{v}_i^\top \mathbf{v}_j + b_i + b_j - \log(X_{ij})$.
       - Вычислите весовой коэффициент $f(X_{ij})$ с использованием выбранной весовой функции.
     - **Обратное распространение ошибки**:
       - Вычислите градиенты:
$$
         \frac{\partial L_{ij}}{\partial \mathbf{v}_i} = 2 f(X_{ij}) E_{ij} \cdot \mathbf{v}_j
$$
$$
         \frac{\partial L_{ij}}{\partial \mathbf{v}_j} = 2 f(X_{ij}) E_{ij} \cdot \mathbf{v}_i
$$
$$
         \frac{\partial L_{ij}}{\partial b_i} = 2 f(X_{ij}) E_{ij}
$$
$$
         \frac{\partial L_{ij}}{\partial b_j} = 2 f(X_{ij}) E_{ij}
$$
     - **Обновление параметров**:
       - Обновите $\mathbf{v}_i$, $\mathbf{v}_j$, $b_i$, $b_j$ с использованием вычисленных градиентов и скорости обучения $\eta$.
       - (Опционально) используйте более продвинутые оптимизаторы, такие как AdaGrad, которые адаптируют скорость обучения для каждого параметра.

6. **Получение конечных векторов**:  
   После завершения обучения, для каждого слова $k$, его окончательное векторное представление может быть получено как $\mathbf{v}_k + \mathbf{v}'_k$ (где $\mathbf{v}'_k$ — контекстный вектор, соответствующий $\mathbf{v}_k$), или просто как $\mathbf{v}_k$.



# 5. Преимущества и недостатки GloVe

## Преимущества

- **Эффективность**:  
  Обучение GloVe относительно быстро по сравнению с некоторыми другими методами, особенно на больших корпусах, поскольку оно работает с агрегированной статистикой со-встречаемости, а не с отдельными контекстами.

- **Использование глобальной статистики**:  
  В отличие от Word2Vec, который фокусируется на локальных контекстах, GloVe явно использует глобальную статистику со-встречаемости, что позволяет ему лучше улавливать общие семантические отношения.

- **Хорошая производительность**:  
  GloVe часто демонстрирует конкурентоспособные результаты на различных задачах NLP, таких как аналогии, классификация и сходство слов.

- **Простота и интерпретируемость**:  
  Функция потерь GloVe относительно проста и интуитивно понятна, что облегчает её понимание и анализ.

## Недостатки

- **Зависимость от матрицы со-встречаемости**:  
  Производительность GloVe сильно зависит от качества и размера матрицы со-встречаемости, которая может быть очень большой для обширных словарей, требуя значительных вычислительных ресурсов для её построения и хранения.

- **Обработка OOV-слов**:  
  Как и многие другие методы, GloVe по умолчанию не может генерировать векторы для слов, отсутствующих в словаре (Out-Of-Vocabulary, OOV). Для этого требуются дополнительные механизмы (например, использование субсловных единиц или предварительно обученных векторов).

- **Чувствительность к гиперпараметрам**:  
  Производительность может быть чувствительна к выбору гиперпараметров, таких как размерность векторов, размер контекстного окна, $x_{\text{max}}$ и $\alpha$ в весовой функции, а также скорость обучения.



# 6. Заключение

Модель GloVe представляет собой мощный и элегантный подход к созданию векторных представлений слов. Объединяя преимущества методов, основанных на частоте встречаемости, и прогнозных моделей, GloVe эффективно кодирует семантические и синтаксические отношения между словами в плотные векторные пространства. Её математическая основа, построенная на минимизации взвешенной квадратичной ошибки между скалярным произведением векторов и логарифмом частоты со-встречаемости, позволяет создавать высококачественные эмбеддинги, которые находят широкое применение в различных задачах обработки естественного языка. Понимание прямого прохода и обратного распространения ошибки является ключом к пониманию того, как модель обучается и адаптирует свои параметры для достижения этой цели.

















# Аналитический пример модели GloVe: Пошаговые вычисления

Этот пример демонстрирует один полный шаг обучения (прямой проход, вычисление градиентов и обновление параметров) для одной пары слов в модели GloVe. Мы будем использовать очень маленький корпус и простые значения для наглядности.



## 1. Исходные данные

### 1.1. Корпус текстов

Предположим, у нас есть очень маленький корпус:
- «cat chases mouse»
- «mouse runs from cat»



### 1.2. Словарь

Из корпуса извлекаем уникальные слова и присваиваем им индексы:
- `cat`: 0  
- `chases`: 1  
- `mouse`: 2  
- `runs`: 3  
- `from`: 4  

Размер словаря $V = 5$.



### 1.3. Матрица со-встречаемости $X$

Используем контекстное окно размером 1 (только непосредственные соседи) с симметричным учётом.

- «cat chases mouse»:
  - (cat, chases): 1
  - (chases, cat): 1
  - (chases, mouse): 1
  - (mouse, chases): 1
- «mouse runs from cat»:
  - (mouse, runs): 1
  - (runs, mouse): 1
  - (runs, from): 1
  - (from, runs): 1
  - (from, cat): 1
  - (cat, from): 1

Суммируя, получаем матрицу со-встречаемости $X$:

| Word \ Context | cat (0) | chases (1) | mouse (2) | runs (3) | from (4) |
|----------------|---------|------------|-----------|----------|----------|
| cat (0)        | 0       | 1          | 0         | 0        | 1        |
| chases (1)     | 1       | 0          | 1         | 0        | 0        |
| mouse (2)      | 0       | 1          | 0         | 1        | 0        |
| runs (3)       | 0       | 0          | 1         | 0        | 1        |
| from (4)       | 1       | 0          | 0         | 1        | 0        |



### 1.4. Инициализация параметров

Предположим, размерность векторов $D = 2$.

Инициализируем векторы слов $\mathbf{v}_i$, контекстные векторы $\mathbf{v}'_j$ и смещения $b_i$, $b_j$ случайными значениями.

#### Векторы слов ($\mathbf{v}_i$):
- $\mathbf{v}_{\text{cat}} = \mathbf{v}_0 = [0.1, 0.2]$  
- $\mathbf{v}_{\text{chases}} = \mathbf{v}_1 = [0.3, 0.4]$  
- $\mathbf{v}_{\text{mouse}} = \mathbf{v}_2 = [0.5, 0.6]$  
- $\mathbf{v}_{\text{runs}} = \mathbf{v}_3 = [0.7, 0.8]$  
- $\mathbf{v}_{\text{from}} = \mathbf{v}_4 = [0.9, 1.0]$  

#### Контекстные векторы ($\mathbf{v}'_j$):
- $\mathbf{v}'_{\text{cat}} = \mathbf{v}'_0 = [0.05, 0.15]$  
- $\mathbf{v}'_{\text{chases}} = \mathbf{v}'_1 = [0.25, 0.35]$  
- $\mathbf{v}'_{\text{mouse}} = \mathbf{v}'_2 = [0.45, 0.55]$  
- $\mathbf{v}'_{\text{runs}} = \mathbf{v}'_3 = [0.65, 0.75]$  
- $\mathbf{v}'_{\text{from}} = \mathbf{v}'_4 = [0.85, 0.95]$  

> **Примечание**: В оригинальной статье GloVe векторы $\mathbf{v}_i$ и $\mathbf{v}'_j$ обучаются независимо. Здесь $\mathbf{v}'_j$ — контекстный вектор для слова $j$.

#### Смещения ($b_i$, $b_j$):
- $b_{\text{cat}} = b_0 = 0.01$  
- $b_{\text{chases}} = b_1 = 0.02$  
- $b_{\text{mouse}} = b_2 = 0.03$  
- $b_{\text{runs}} = b_3 = 0.04$  
- $b_{\text{from}} = b_4 = 0.05$  

#### Гиперпараметры:
- Скорость обучения: $\eta = 0.01$  
- Параметры весовой функции:
  - $x_{\text{max}} = 10$
  - $\alpha = 0.75$



## 2. Выбор пары для обучения

Выберем пару слов $(i, j)$ для демонстрации одного шага обучения:  
$(\text{cat}, \text{chases})$, то есть $i = 0$, $j = 1$.  
Из матрицы: $X_{01} = 1$.



## 3. Прямой проход (Forward Pass)

### 3.1. Извлечение параметров для $(i=0, j=1)$
- $\mathbf{v}_0 = [0.1, 0.2]$  
- $\mathbf{v}'_1 = [0.25, 0.35]$  
- $b_0 = 0.01$  
- $b_1 = 0.02$  
- $X_{01} = 1$



### 3.2. Вычисление предсказанного значения

$$
\text{prediction} = \mathbf{v}_0^\top \mathbf{v}'_1 + b_0 + b_1
$$

$$
\mathbf{v}_0^\top \mathbf{v}'_1 = (0.1 \cdot 0.25) + (0.2 \cdot 0.35) = 0.025 + 0.070 = 0.095
$$

$$
\text{prediction} = 0.095 + 0.01 + 0.02 = 0.125
$$



### 3.3. Вычисление целевого значения

$$
\text{target} = \log(1 + X_{01}) = \log(1 + 1) = \log(2) \approx 0.6931
$$



### 3.4. Вычисление ошибки

$$
\text{error} = \text{prediction} - \text{target} = 0.125 - 0.6931 = -0.5681
$$


### 3.5. Вычисление весового коэффициента $f(X_{01})$

Поскольку $X_{01} = 1 < x_{\text{max}} = 10$:

$$
f(X_{01}) = \left( \frac{X_{01}}{x_{\text{max}}} \right)^\alpha = \left( \frac{1}{10} \right)^{0.75} = 0.1^{0.75} \approx 0.1778
$$



### 3.6. Вычисление взвешенной квадратичной ошибки $L_{01}$

$$
L_{01} = f(X_{01}) \cdot (\text{error})^2 = 0.1778 \cdot (-0.5681)^2
$$

$$
(-0.5681)^2 = 0.32276
$$

$$
L_{01} = 0.1778 \cdot 0.32276 \approx 0.05747
$$



## 4. Обратное распространение ошибки (Backpropagation)

Общая формула для градиента по параметру $P$:

$$
\frac{\partial L_{ij}}{\partial P} = 2 f(X_{ij}) \cdot \text{error} \cdot \frac{\partial (\text{prediction})}{\partial P}
$$

Мы уже знаем:
- $f(X_{01}) \approx 0.1778$
- $\text{error} \approx -0.5681$

Общий множитель:
$$
2 \cdot f(X_{01}) \cdot \text{error} = 2 \cdot 0.1778 \cdot (-0.5681) \approx -0.2023
$$



### 4.1. Градиент по $\mathbf{v}_0$ (вектор слова «cat»)

$$
\frac{\partial L_{01}}{\partial \mathbf{v}_0} = 2 f(X_{01}) \cdot \text{error} \cdot \mathbf{v}'_1 = -0.2023 \cdot [0.25, 0.35]
$$

$$
= [-0.2023 \cdot 0.25,\ -0.2023 \cdot 0.35] = [-0.050575,\ -0.070805]
$$



### 4.2. Градиент по $\mathbf{v}'_1$ (контекстный вектор слова «chases»)

$$
\frac{\partial L_{01}}{\partial \mathbf{v}'_1} = 2 f(X_{01}) \cdot \text{error} \cdot \mathbf{v}_0 = -0.2023 \cdot [0.1, 0.2]
$$

$$
= [-0.2023 \cdot 0.1,\ -0.2023 \cdot 0.2] = [-0.02023,\ -0.04046]
$$



### 4.3. Градиент по $b_0$ (смещение слова «cat»)

$$
\frac{\partial L_{01}}{\partial b_0} = 2 f(X_{01}) \cdot \text{error} \cdot 1 = -0.2023
$$



### 4.4. Градиент по $b_1$ (смещение слова «chases»)

$$
\frac{\partial L_{01}}{\partial b_1} = 2 f(X_{01}) \cdot \text{error} \cdot 1 = -0.2023
$$



## 5. Обновление параметров

Используем скорость обучения $\eta = 0.01$.



### 5.1. Обновление $\mathbf{v}_0$

$$
\mathbf{v}_0^{\text{new}} = \mathbf{v}_0^{\text{old}} - \eta \cdot \frac{\partial L_{01}}{\partial \mathbf{v}_0}
$$

$$
= [0.1, 0.2] - 0.01 \cdot [-0.050575, -0.070805]
$$

$$
= [0.1 + 0.00050575,\ 0.2 + 0.00070805] = [0.10050575,\ 0.20070805]
$$



### 5.2. Обновление $\mathbf{v}'_1$

$$
\mathbf{v}'_1^{\text{new}} = \mathbf{v}'_1^{\text{old}} - \eta \cdot \frac{\partial L_{01}}{\partial \mathbf{v}'_1}
$$

$$
= [0.25, 0.35] - 0.01 \cdot [-0.02023, -0.04046]
$$

$$
= [0.25 + 0.0002023,\ 0.35 + 0.0004046] = [0.2502023,\ 0.3504046]
$$



### 5.3. Обновление $b_0$

$$
b_0^{\text{new}} = b_0^{\text{old}} - \eta \cdot \frac{\partial L_{01}}{\partial b_0}
$$

$$
= 0.01 - 0.01 \cdot (-0.2023) = 0.01 + 0.002023 = 0.012023
$$



### 5.4. Обновление $b_1$

$$
b_1^{\text{new}} = b_1^{\text{old}} - \eta \cdot \frac{\partial L_{01}}{\partial b_1}
$$

$$
= 0.02 - 0.01 \cdot (-0.2023) = 0.02 + 0.002023 = 0.022023
$$



## 6. Заключение по примеру

Мы выполнили один шаг обновления параметров для пары слов $(\text{cat}, \text{chases})$ в модели GloVe. В реальном обучении этот процесс повторяется для всех пар с ненулевой частотой со-встречаемости в течение многих эпох, пока функция потерь не сойдётся к минимуму. С каждым шагом векторы и смещения будут постепенно корректироваться, чтобы лучше отражать статистические отношения между словами в корпусе.

Этот пример демонстрирует, как каждый компонент функции потерь и её градиентов влияет на обновление параметров, приближая скалярное произведение векторов к логарифму частоты их со-встречаемости.

